In [ ]:
import rps.robotarium as robotarium
from rps.utilities.transformations import *
from rps.utilities.barrier_certificates import *
from rps.utilities.misc import *
from rps.utilities.controllers import *
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

import scipy.stats as st
import time

## Introduction

The aim of the project is to solve a Forward Optimal Control problem and then reconstruct its cost function by means of Inverse Optimal Control.
#### Forward Optimal Control
The aim of the FOC is to allow unicycle robots to navigate in an unstructured environment filled with obstacles. The robots should reach the goal position while avoiding the obstacles. 

Some hypotheses are made:
- The robots are equipped with sensors that allow to know their position in the environment at each time step, but this information could be subject to noise.
- The position of the obstacles and the goal in the environment are known at each time step.

The control algorithm is fully probabilistic and it works without knowing the dynamical model of the robot. 

#### Inverse Optimal Control
The aim of the IOC is to reconstruct the cost function of the FOC starting from the trajectories of the robots as well as the control inputs. In this case we assume to know the position of the goal but we are not aware of the position of the obstacles.

Both the algorithms are validated through the API of the Robotarium platform by Georgia Tech, which allows to simulate the robots in the environment and to visualize the results.

Below some variables and functions that will be used in the following are defined:

In [ ]:
# Defining the input axes
# Note: the boundaries for the control actions are the actuation constraints imposed by the physics of the robots

control_space_size = 3  # Three possible inputs for each control axis

U_space_1 = np.array(np.linspace((-0.5),(0.5),control_space_size)) # Control space for the first axis
U_space_2 = np.array(np.linspace((-0.5),(0.5),control_space_size)) # Control space for the second axis
time_step = 0.033 # Robotarium time-step (from the documentation)

In [ ]:
# This function performs a "model" step using the documented dynamics
# Note: from the viewpoint of the controller the dynamics is not necesarily known
def model_step(x,velocities,time_step):
    """This function calculates the next pose of the robots based on the current pose, the velocities and the simulation time-step.
    Args:
        x : The current position of the robot. 2x1 column vector .
        velocities : The velocities of the robot along the two control axes. 2x1 column vector. 
        time_step : simulation time-step
    Returns:
        The next pose of the robot as a 2x1 column vector.
    """
    poses = np.zeros((2,1))
    # Update pose of the robots
    poses[0] = x[0] + time_step*velocities[0]
    poses[1] = x[1] + time_step*velocities[1]
    return(poses)

#Get the value of a Gaussian pf at a given point x, with mean u and covariance covar
def my_logpdf(x, u, covar):
    """This function calculates the value of a multivariate Gaussian pdf at a given point x, with mean u and covariance covar

    Args:
        x : The point at which the pdf is evaluated. Nx1 column vector.
        u : The mean vector of the Gaussian distribution. Nx1 column vector.
        covar : The covariance matrix of the Gaussian distribution. NxN matrix.

    Returns:
        The value of the Gaussian pdf at x.
    """
    k = len(x)  # dimension
    a = np.transpose(x - u)
    b = np.linalg.inv(covar)
    c = x - u
    d = np.matmul(a, b)
    e = np.matmul(d, c)
    numer = np.exp(-0.5 * e)
    f = (2 * np.pi)**k
    g = np.linalg.det(covar)
    denom = np.sqrt(f * g)
    pdf = numer / denom
    return pdf

In [ ]:
# Define goal and obstacle points
goal_points = np.array(np.mat('-1.4; -0.8; 0'))

obs_points = np.array(np.mat('0 0 0 0 0;0.2 0.4 0.6 0.8 -0.8;0 0 0 0 0'))

# WP0 - Problem statements and cost function analysis

### Forward Control Problem Statement
In the most general case, the Forward Optimal Control problem is defined as follows:\
\
\
Given the joint probability function $q_{0:N} := q_0 (\mathbf x_0)\prod^N_{k=1} q^{(x)}_k(\mathbf x_k \mid \mathbf u_k, \, \mathbf x_{k-1}) q^{(u)}_k(\mathbf u_k \mid \mathbf  x_{k-1})$ find the sequence of probability functions $\left\{{p^{(u)}_{k\mid k-1}}^*\right\}_{1:N}$ such that:

$$
\begin{equation}
\left\{p^{(u) \quad *}_{k|k-1}\right\}_{1:N} \in \argmin_{\{p^{(u)}_{k|k-1}\}_{1:N}} \left\{\mathcal D_{KL}(p_{0:N} || q_{0:N}) + \sum^N_{k=1}\Bbb E_{\overline p_{k-1:k}}\left[\Bbb E_{p^{(x)}_{k \mid k-1}}[c_k(\mathbf{X}_k)]\right] \right\}
\end{equation}
$$

$$
\begin{align*}
s.t. \ \ p^{(u)}_{k\mid k-1} \in \mathcal D \ \ \forall k \in \mathcal T
\end{align*}
$$

$ \text {where } \ \  \overline p_{k-1 \mid k} := p_{k-1}(\mathbf x_{k-1},\mathbf u_k) $.\
\
\
In our case the problem can be simplified because:

- The first term in the cost functional acts as a regularzer that biases the behavior of the system towards the reference distribution $q_{0:N}$. In our case the reference pf $q_{0:N}$ is uniform so the first term becomes an entropic regularizer;
- The cost $c_k(\mathbf{X}_k)=c(\mathbf{X}_k)$ is stationary.

Thus the formulation of the forward control problem becomes:\
\
\
Find the sequence of probability distributions $\left\{{p^{(u)}_{k\mid k-1}}^*\right\}_{1:N}$ such that:

$$
\begin{equation}
\left\{p^{(u) \quad *}_{k|k-1}\right\}_{1:N} \in \argmin_{\{p^{(u)}_{k|k-1}\}_{1:N}} \left\{ -H(p_{0:N}) + \sum^N_{k=1}\Bbb E_{\overline p_{k-1:k}}\left[\Bbb E_{p^{(x)}_{k:k-1}}[c(
\mathbf{X}_k)]\right] \right\}
\end{equation}
$$
$$
\begin{align*}
\\ s.t. \ \ p^{(u)}_{k\mid k-1} \in \mathcal D \ \ \forall k \in \mathcal T
\end{align*}
$$
$ \text {where } \ \  \overline p_{k-1 \mid k} := p_{k-1}(\mathbf x_{k-1},\mathbf u_k) \ \text{ and } \ H(p_{0:N}) \text{ is the entropy of the probability function } p_{0:N}$.

### Inverse Control Problem Statement
In order to define the Inverse Control Problem we define the following assumptions:
- We suppose that there exists a sequence of pfd $\left\{{p^{(u)}_{k\mid k-1}}^*\right\}_{1:N}$ that is feasible for the forward control problem defined above and such that the objective functional is finite;
- We suppose that the cost-to-go is expressed as a linear combination of some features $\mathbf{h}(\mathbf x_k)$. In other words $\overline c_k(\mathbf x_k)=-\mathbf w_k^T \mathbf h(\mathbf x_k)$ where $\mathbf h(\mathbf x_k):=[h_1(\mathbf x_k),\dots, h_F(\mathbf x_k)]^T$ is the features vector and $h_i: X\rarr \Bbb  R$ are known functions with $i \in \{1,\dots,F\}$ and $\mathbf w_k:=[w_{k,1},\dots,w_{k,F}]^T$ is the weight vector;

The inverse control problem (already formulated in the case of a stationary cost and uniform $q_{0:N}$) is defined as follows:

Let $\{(\hat{\mathbf x}_{k-1}, {\hat {\mathbf u}_{k}})\}_{1:M}$ be the observed data with $\hat{\mathbf x}_{k-1} \sim p^{(x)}_{k\mid k-1}, \hat {\mathbf u}_{k} \sim p^{(u)}_{k\mid k-1}$. The maximum likelihood estimate for the cost

$$ \mathbf w^* := [w_{1}^{*T}, \cdots, \mathbf w_{M}^{*T}]^{T} \in 
\argmin \left\{ \sum_{i=1}^{M}\left(-\Bbb E_{p(x_{k}|\hat x_{k-1}, \hat u_k)}\left[w_{k}^{T} \cdot h(x_{k})\right]
+ 
\ln\left(\sum_{u_k} \exp\left(-\Bbb E_{p(x_{k}|\hat x_{k-1}, u_k)} \left[\ln(p(x_{k}|\hat x_{k-1}, u_k))+\mathbf w^{T} \cdot h(x_{k})\right]\right)\right)\right) \right\} $$

### Cost function analysis

In [ ]:
##### WP0: formalize the control problem #####

# Task: reverse engineer the cost function used by the robots. What is the problem formulation? 
#      Is the one below a good cost for the task? Create a heatmap to visualize the cost 
"""The task has been answered in the blocks below."""

def state_cost(state,goal_points,obs_points):
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum + 10*(np.exp(-0.5*((state[0]-(-1.5))/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/0.02)**2)/(0.02*np.sqrt(2*np.pi)))
    return(cost)

Let $\mathbf x_k = [x_k, y_k] \in \Bbb R^2$ be the current position of the robot, $\mathbf x_d \in \Bbb R^2$ be the desired goal for the robot, $\mathbf o_i \in \Bbb R^2$ be the position of the i-th obstacle, n the number of obstacles. The cost function that is subject to minimization is:

$$
\begin{equation}
    c(\mathbf{x}_k) = 30(\mathbf x_k - x_d)^2 + 20\sum_{i=1}^n g_i(\mathbf x_k) + 10\sum_{j=1}^4 h_j(\mathbf x_k)
\end{equation}
$$
where:
$$
g_i(\mathbf{x}_k) = \frac{1}{\sqrt{(2\pi)^2\text{det}({\mathbf {\Sigma}}_o)}} \exp\left(-\frac{1}{2}(\mathbf x_k - \mathbf o_i)^T \Sigma_o^{-1} (\mathbf x_k - \mathbf o_i)\right)
$$
is a bivariate gaussian function with mean vector $\mu=\mathbf o_i$ and covariance matrix $\mathbf \Sigma_o = \begin{bmatrix} 0.02 & 0 \\ 0 & 0.02 \end{bmatrix}$ and
$$
h_1(\mathbf x_k) = \frac{1}{(0.02) \sqrt{2\pi}} \exp\left(-\frac{1}{2}\left(\frac{x_k - (-1.5)}{(0.02)}\right)^2\right) \\
h_2(\mathbf x_k) = \frac{1}{(0.02) \sqrt{2\pi}} \exp\left(-\frac{1}{2}\left(\frac{x_k - (1.5)}{(0.02)}\right)^2\right) \\ 
h_3(\mathbf x_k) = \frac{1}{(0.02) \sqrt{2\pi}} \exp\left(-\frac{1}{2}\left(\frac{y_k - (-1)}{(0.02)}\right)^2\right) \\ 
h_4(\mathbf x_k) = \frac{1}{(0.02) \sqrt{2\pi}} \exp\left(-\frac{1}{2}\left(\frac{y_k - (1)}{(0.02)}\right)^2\right) \\
$$
are four univariate gaussian functions with variance $\sigma^2 = 0.02$. The gaussians $h_1$ and $h_2$ are calculated with respect to the x-axis and have means $\mu_1=-1.5$, $\mu_2=1.5$. The gaussians $h_3$ and $h_4$ are calculated with respect to the y-axis and have means $\mu_3=-1$, $\mu_4=1$. 

Given the task to be performed, the cost function can be interpreted as follows:
- The first term $(\mathbf x_k - x_d)^2$ is ellyptic paraboloid whose global minimum is centered in the goal position $\mathbf x_d$. This term is used to push the robots towards the goal.
- Each obstacle is modeled by a bivariate gaussian function $g_i$ centered in the position of the obstacle $\mathbf o_i$. This term has a maximum on the obstacle and is used to push the robots away from the obstacles.
- Each wall is represented by a gaussian function $h_j$. The gaussians $h_1$ and $h_2$ are centered in the x-coordinates of the left and right walls respectively. The gaussians $h_3$ and $h_4$ are centered in the y-coordinates of the bottom and top walls respectively. These terms are used to push the robots away from the walls.
- The weights of the cost function are chosen in order to obtain a balance between the terms.

Below we provide some visualizations of the cost function.


#### Cost function heatmap visualization
The code below creates an heatmap of the cost function.

In [ ]:
# Define the grid boundaries
X_LIMITS = [-1.5, 1.5]
Y_LIMITS = [-1, 1]
EPSILON_LIMITS = 0 # Used to plot a little bit outside the robotarium boundaries. 0 stans for disabled.
X_SIZE = X_LIMITS[1]-X_LIMITS[0]+2*EPSILON_LIMITS
Y_SIZE = Y_LIMITS[1]-Y_LIMITS[0]+2*EPSILON_LIMITS

# Define the grid of points where to evaluate the cost function
X_axis = np.linspace(X_LIMITS[0]-EPSILON_LIMITS, X_LIMITS[1]+EPSILON_LIMITS, 300)
Y_axis = np.linspace(Y_LIMITS[0]-EPSILON_LIMITS, Y_LIMITS[1]+EPSILON_LIMITS, 200)

X_grid, Y_grid = np.meshgrid(X_axis, Y_axis)

# Evaluate the cost function on the grid
costs = np.array([[state_cost((x,y),goal_points,obs_points) for x in X_axis] for y in Y_axis] )
costs = costs.squeeze()

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot()
ax.set_xlabel('X [m]', labelpad=20)
ax.set_ylabel('Y [m]', labelpad=20)
ax.set_title('Cost function', fontdict={'fontsize': 20})

# Plot the heatmap of the cost function
pcolormesh = ax.pcolormesh(X_grid, Y_grid, costs, cmap='coolwarm', shading='auto', zorder=1)

# Plot the level curves of the cost function
contour = ax.contour(X_grid, Y_grid, costs, cmap='coolwarm', levels=10, linewidths=2, linestyles='dashed', zorder=2, alpha=1, vmin=100, vmax=450) #vmin and vmax are used to make the contour lines more visible

# Scatter the obstacle points
ax.scatter(obs_points[0,:], obs_points[1,:], marker='o', s=50, color='red', zorder=3)

# Add a colorbar
fig.colorbar(pcolormesh, ax=ax) 

# Create a rectangle
rectangle = Rectangle((X_LIMITS[0], Y_LIMITS[0]), X_LIMITS[1]-X_LIMITS[0], Y_LIMITS[1]-Y_LIMITS[0], fill=False, color="black", linewidth=1, zorder=3)
# Add the rectangle to the plot
ax.add_patch(rectangle);


#### Cost function 3D visualization
The code below creates a 3D visualization of the cost function. The visualization is interactive and allows to rotate the cost function in order to have a better understanding of its shape.

In [ ]:
%matplotlib tk
from matplotlib import cm

plt.rcParams.update({'font.size': 12})

# Interpolate the cost function to plot it as a surface
#from scipy.interpolate import griddata
#dense_x = np.linspace(X_LIMITS[0]-EPSILON_LIMITS, X_LIMITS[1]+EPSILON_LIMITS, 300)
#dense_y = np.linspace(Y_LIMITS[0]-EPSILON_LIMITS, Y_LIMITS[1]+EPSILON_LIMITS, 200)
#dense_X, dense_Y = np.meshgrid(dense_x, dense_y)
#interpolated_costs = griddata((X.flatten(),Y.flatten()), costs.flatten(), (dense_X.flatten(), dense_Y.flatten()), method='cubic').reshape(dense_X.shape)


# Plot the cost function
fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(projection='3d')
ax.set_xlabel('X [m]', labelpad=10)
ax.set_ylabel('Y [m]', labelpad=10)
ax.set_title('Cost function', fontdict={'fontsize': 20})

surf = ax.plot_surface(X_grid, Y_grid, costs, cmap=cm.coolwarm, linewidth=0, antialiased=False, ccount=100, rcount=100) #rcount and ccount are used to make the surface smoother
fig.colorbar(surf, shrink=0.5);

#### Cost function gradient module visualization
The code below creates a heatmap visualization of the module of the gradient of the cost function. This visualization is useful to understand the behavior of the robots in the environment.

In [ ]:
%matplotlib inline
# Calculate the gradient of the cost function
X_grad, Y_grad = np.gradient(costs.T, X_axis, Y_axis) #We need to transpose the costs matrix because we are passing the x axis as the first argument
# Calculate the module of the gradient
Z_grad = np.sqrt(X_grad**2 + Y_grad**2).T #We need to transpose the result because we need to plot it

fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot()
ax.set_xlabel('X [m]', labelpad=20)
ax.set_ylabel('Y [m]', labelpad=20)
ax.set_title('Cost function gradient module', fontdict={'fontsize': 20})

# Plot the heatmap of the module of the gradient of the cost function
pcolormesh = ax.pcolormesh(X_grid, Y_grid, Z_grad, zorder=1, cmap='RdGy', vmin=0, vmax=150)

# Add a colorbar
fig.colorbar(pcolormesh, ax=ax, extend='max');

# WP1

In [ ]:
##### WP1: fill-in the code for the function below.
#         The function needs to return the optimal action sampled from the optimal policy.
#         The action is used in the simulation loop #####

def Control_step(state,U_space_1,U_space_2,goal_points,obs_points):
    ###
    # Perform a control step given the fact that the target pf is uniform.
    # The function first gets the target pf (uniform) and then applies the control solution we saw in class
    
    target_pf = 1/control_space_size**2 # Uniform pf q(u_k|x_k-1)
    time_step = 0.033 # The Robotarium time-step

    pf = np.zeros((control_space_size,control_space_size)) #Initialize pf
    for i in range(control_space_size):
        for j in range(control_space_size):
            # Task: what do the next three lines do?
            """
            The next three lines calculate the probability function p(x_k|u_k,x_{k-1}), which is the probabilistic description of the robot dynamics.
            The robot dynamics is assumed to be x_k = x_{k-1} + u_k * time_step, where u_k is the velocity control action.
            Here we are emulating a Gaussian measurement noise for the robot position, hence we choose the transition probability function p(x_k|u_k,x_{k-1})
            to be a Gaussian distribution with mean x_{k-1} + u_k * time_step and covariance matrix cov = [[0.001, 0.0002], [0.0002, 0.001]].
            """
            next_state = model_step(state,[U_space_1[i],U_space_2[j]],time_step) # The mean of the probability function p(x_k|u_k,x_{k-1})
            cov = np.array([[0.001, 0.0002], [0.0002, 0.001]]) # The covariance matrix of the probability function p(x_k|u_k,x_{k-1})
            f = st.multivariate_normal(next_state.reshape((2,)),cov) #f is the probability function p(x_k|u_k,x_{k-1})
            # Task: what do the next two lines do?
            """
            The next two lines extract 20 samples from the probability function p(x_k|u_k,x_k-1) calculated before.
            """
            N_samples = 20
            next_sample = f.rvs(N_samples) 

            # Task: what do the next three lines do?
            """
            The next three lines calculate an estimation of the expectation of the cost function c(X_k) over the pf p(x_k|u_k,x_k-1).
            The estimation is calcualated by means of Monte Carlo sampling, as a mean of the cost function evaluated on the 20 samples extracted before.
            According to the law of large numbers, the more samples we extract, the more accurate the estimation will be.
            The estimated expectation of the cost function is calculated because we need it to calculate the greedy approximation of the optimal policy.
            """
            cost=0
            for k in range(N_samples):
                cost += state_cost(next_sample[k,:],goal_points,obs_points)/N_samples
            # Task: write here a line of code, defining the variable log_DKL that contains the exponential in the policy
            """
            Here we implement the greedy sub-optimal solution for the FOC problem as we only consider the immediate cost c(X_k).
            The expectation of the cost function is already calculated above.
            We exploit the fact that p(x_k|u_k,x_{k-1}) is a Gaussian distribution with known mean and covariance matrix to calculate the DKL
            analytically, without making numerical estimations. 
            More specifically we can use the entropy because it is defined as the expectation of the negative logarithm of the pf w.r.t. the pf itself.
            """
            log_DKL = np.exp(f.entropy()-cost).item() # item() function is called to cast the ndarray to a scalar which avoids a warning in the code that will follow
            pf[i,j] = log_DKL # Calculate the DKL for each possible input, get corresponding probability
    # Task: obtain the normalizer for the policy, call it S2
    """
    The normalizer for the policy is the exponential twisted kernel.
    Here we calculate it simply as the sum of the pf calculated on each possible input.
    """
    S2 = np.sum(pf)
    # Task: obtain the normalized pf (call the variable pf)
    """
    To normalize the pf, we divide it by the normalizer S2.
    """
    pf = np.divide(pf,S2)

    """Now that we calculated the greedy policy we can sample the next control action from it."""
    # This is a trick to properly sample from the multi-dimensional pf
    flat = pf.flatten()
    sample_index = np.random.choice(a=flat.size, p=flat) # We sample an action from the flattened pf
    # Take this index and adjust it so it matches the original array
    adjusted_index = np.unravel_index(sample_index, pf.shape) # This function converts the index of the flattened pf back into a tuple of two indices for the original action space

    action = np.reshape(np.array([U_space_1[adjusted_index[0]],U_space_2[adjusted_index[1]]]),(2,1)) # We extract the control actions on the two control axes and we reshape them into a 2x1 column vector

    return(action)

### Discussion on the solution of the FOC problem
The optimal solution to the FOC problem we defined before (with entropy regularization and a stationary cost) is given by the following equation:

$${p_{k|k-1}^{(u)}}^* = \frac{\exp\left(-\mathbb{E}_{p_{k|k-1}^{(x)}}\left[\ln(p_{k|k-1}^{(x)}) + \overline{c}(\mathbf{X}_k)\right]\right)}{\sum_{u_k} \exp\left(-\mathbb{E}_{p_{k|k-1}^{(x)}}\left[\ln(p_{k|k-1}^{(x)}) + \overline{c}(\mathbf{X}_k)\right]\right)}$$
where $\overline{c}(\mathbf{X}_k) = c(\mathbf X_k) - \hat c(\mathbf X_k)$ and $\hat c_k(\mathbf X_k)$ calculated with the recursive formula:

$$\hat c(\mathbf X_k) = \ln \left(\sum_{\mathbf u_k}\left(-\Bbb E_{p_{k|k+1}^{(x)}}\left[\ln(p_{k|k+1}^{(x)})+\overline c(\mathbf X_{k+1})\right]\right)\right),\\
\Bbb E_{p_{N+1|N}^{(x)}}\left[\ln(p_{N+1|N}^{(x)})+\overline c(\mathbf X_{N+1})\right] = 0$$

In order to reduce the computational complexity in the code above we implemented a sub-optimal solution to the FOC problem by using the immediate cost instead of the cost-to-go:
$${p_{k|k-1}^{(u)}} = \frac{\exp\left(-\mathbb{E}_{p_{k|k-1}^{(x)}}\left[\ln(p_{k|k-1}^{(x)}) + {c}(\mathbf{X}_k)\right]\right)}{\sum_{u_k} \exp\left(-\mathbb{E}_{p_{k|k-1}^{(x)}}\left[\ln(p_{k|k-1}^{(x)}) + {c}(\mathbf{X}_k)\right]\right)}$$

This means that in order to evaluate the control action at each time step the agent only looks at a single step ahead. This could lead to the agent getting stuck in a local minimum of the cost function.

### Comparison with the methods seen in class
The general formulation of the FOC problem is a generalization of the so-called "fully probabilistic control" 

# WP2

In [ ]:
#WP2: Simulate (4 experiments) and visualize each robot's trajectory

In [ ]:
%matplotlib tk
# Instantiate Robotarium object
N = 1 # Amount of robots per simulation
MAX_ITERATIONS = 5000 # Maximum number of iterations per experiment beceause the robot is not guaranteed to arrive at the goal
# Initial conditions of the robot for 4 experiments
initial_conditions = [np.array(np.mat('1.4;0.9; 0')),np.array(np.mat('0.2;0.9; 0')),np.array(np.mat('1.2;-0.5; 0')),np.array(np.mat('-1;0.9; 0'))] #Initial pose of the robots


N_experiment = 4
# X_si is going to be two-dimensional state history
X_Si = [0]*N_experiment
# D_Xi is going to be two-dimensional inputs history
D_Xi = [0]*N_experiment

# This first for loop creates the initial conditions
for I in range(N_experiment):

    X_si = []
    D_xi = []
    #we use mp.copy to avoid that the initial conditions are modified by the simulation
    r = robotarium.Robotarium(number_of_robots=N, show_figure=True, initial_conditions=initial_conditions[I], sim_in_real_time=False)

    # Create mapping from the control inputs to the actual velocity commands to the unicycle
    # Note: this is a very practical situation (robots often provide transformation functions to low level commands)
    si_to_uni_dyn = create_si_to_uni_dynamics_with_backwards_motion() #Converts single integrator inputs to unicycle inputs (low-level controller)
    _, uni_to_si_states = create_si_to_uni_mapping()
    
    # define x initially
    x = r.get_poses()
    x_si = uni_to_si_states(x)

    # Plotting Parameters
    CM = np.random.rand(N+10,3) # Random Colors
    goal_marker_size_m = 0.15
    obs_marker_size_m = 0.15
    marker_size_goal = determine_marker_size(r,goal_marker_size_m)
    marker_size_obs = determine_marker_size(r,obs_marker_size_m)
    font_size = determine_font_size(r,0.1)
    line_width = 5

    # Create Goal Point Markers
    #Text with goal identification
    goal_caption = ['G{0}'.format(ii) for ii in range(goal_points.shape[1])]
    #Plot text for caption
    goal_points_text = [r.axes.text(goal_points[0,ii], goal_points[1,ii], goal_caption[ii], fontsize=font_size, color='k',fontweight='bold',horizontalalignment='center',verticalalignment='center',zorder=-2)
    for ii in range(goal_points.shape[1])]
    goal_markers = [r.axes.scatter(goal_points[0,ii], goal_points[1,ii], s=marker_size_goal, marker='s', facecolors='none',edgecolors=CM[ii,:],linewidth=line_width,zorder=-2)
    for ii in range(goal_points.shape[1])]

    #Text with goal identification
    obs_caption = ['OBS{0}'.format(ii) for ii in range(obs_points.shape[1])]
    #Plot text for caption
    obs_points_text = [r.axes.text(obs_points[0,ii], obs_points[1,ii], obs_caption[ii], fontsize=font_size, color='k',fontweight='bold',horizontalalignment='center',verticalalignment='center',zorder=-2)
    for ii in range(obs_points.shape[1])]
    obs_markers = [r.axes.scatter(obs_points[0,ii], obs_points[1,ii], s=marker_size_obs, marker='s', facecolors='none',edgecolors=CM[ii+1,:],linewidth=line_width,zorder=-2)
    for ii in range(obs_points.shape[1])]

    r.step()

    i = 0
    # While the robot is away from the objective ...
    while (np.size(at_pose(np.vstack((x_si,x[2,:])), goal_points, position_error=0.15,rotation_error=100)) != N and i < MAX_ITERATIONS ):

        # Get poses of agents
        x = r.get_poses()
        x_si = uni_to_si_states(x)

        #Add to the dataset
        X_si.append(x_si)

        # The lines below define the pdf of the robot 
        cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
        x_pdf = st.multivariate_normal(x_si.reshape((2,)),cov)
        x_sample = x_pdf.rvs() #Noisy state

        # This is about plotting
        for j in range(goal_points.shape[1]):
            goal_markers[j].set_sizes([determine_marker_size(r, goal_marker_size_m)])

        for j in range(obs_points.shape[1]):
            obs_markers[j].set_sizes([determine_marker_size(r, obs_marker_size_m)])

        # Task: compute the action from the policy. Call the variable dxi: 
        # this is the action sampled from the optimal solution to the control problem
        """Here we simply call the function Control_step defined above to calculate the action sampled from the solution to the FOC problem."""
        dxi = Control_step(x_sample, U_space_1, U_space_2, goal_points, obs_points)
        D_xi.append(dxi)

        # Transform single integrator velocity commands to unicycle inputs (low level controller)
        dxu = si_to_uni_dyn(dxi, x)

        # Set the velocities inputs
        r.set_velocities(np.arange(N), dxu)
        # Iterate the simulation
        r.step()
        
        i += 1

    """Here we print an error message if the robot does not reach the goal within the maximum number of iterations."""
    if i >= MAX_ITERATIONS:
        print("SIMULATION {} FAILED!".format(I))
    
    D_Xi[I] = D_xi
    X_Si[I] = X_si

    #Call at end of script to print debug information and for your script to run on the Robotarium server properly
    r.call_at_scripts_end()
    plt.close() # Added to avoid a bug later

In [ ]:
XX = X_Si #Insieme degli stati nel sistema single-integrator (y_1, y_2) per ogni simulazione
UU = D_Xi #Insieme delle azioni di controllo per ogni simulazione

In [ ]:
#Prepare data for plotting
X = []
X_plot = []
U = []
U_plot = []

for i in range(len(XX)):
    X.append(np.array(XX[i])) #X=XX ma è un array di array numpy
    X_plot.append(np.array(XX[i])) #X_plot = XX ma è un array numpy
X = np.concatenate(X, axis=0) #X adesso è un array numpy. La prima dimensione contiene le traiettorie di tutti gli esperimenti concatenate e la secoda indicizza lo stato
X = np.reshape(X, (-1, 2)) # Rimuove l'ultima dimensione inutile
U = []
for i in range(len(UU)):
    U.append(np.array(UU[i])) #U=UU ma è un array di array numpy
    U_plot.append(np.array(UU[i])) #U_plot=UU ma è un array di array numpy

U = np.concatenate(U, axis=0) #Stessa cosa di prima per le traiettorie di ingresso
U = np.reshape(U, (-1, 2))

In [ ]:
%matplotlib inline
#Task: plot trajectories with different colors
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np

plt.figure(figsize=(15,10))

for I in range(N_experiment):
    trajectory = np.array(X_Si[I])
    plt.plot(trajectory[:,0],trajectory[:,1])

#Add a star to the starting point of each trajectory
for I in range(N_experiment):
    trajectory = np.array(X_Si[I])
    plt.scatter(trajectory[0,0], trajectory[0,1], marker='*', s=200, color='black', zorder=3)

#Add a legend to the plot
legend = []
for i in range(N_experiment):
    legend = legend + ['Experiment {}'.format(i+1)]
    plt.legend(legend)

#Draw obstacles
square1 = plt.Rectangle((-1.6,-1), 0.4, 0.4, fc='green',ec="black")
square3 = plt.Rectangle((-0.2,-1), 0.4, 0.4, fc='red',ec="black")
square2 = plt.Rectangle((-0.2,0), 0.4, 0.8, fc='red',ec="black")
plt.gca().add_patch(square2)
plt.gca().add_patch(square1)
plt.gca().add_patch(square3)
plt.ylim(-1,1)
plt.xlim(-1.5,1.5)
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.savefig('Training_Trajectories.jpg',dpi=1000,bbox_inches ='tight')


In rosso si vedono i due ostacoli fisici, uno centrato in (0, -0.8) e un altro più lungo centrato in (0, 0.5).
In verde si vede approssimativamente il goal point, che nella funzione costo è una forma quadratica centrata in (-1.4, -0.8)

# WP3 

### Feature functions visualization

In [ ]:
##### WP3: Reverse engineer the features and visualize them #####

In [ ]:

#Redefining the feature points on the robotarium grid
obs_points_f = np.array(np.mat('0 0 0 0 0 0.8 0.8 0.8 0.8 0.8 -0.8 -0.8 -0.8 -0.8 -0.8;-0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8;0 0 0 0 0 0 0 0 0 0 0 0 0 0 0'))



In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot()
ax.scatter(obs_points_f[0,:],obs_points_f[1,:], color='red')
ax.scatter(goal_points[0,:],goal_points[1,:], color='green')
ax.set_ylim(-1,1)
ax.set_xlim(-1.5,1.5)
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_title('Feature points', fontdict={'fontsize': 20})

In [ ]:
# Task: reverse engineer the features and critically discuss them

N_feature = np.size(obs_points_f,axis=1)+1 #16


"""
There are 16 features:
- The first one models the goal point.
- Another 15 model potential obstacles.

From what the 'feature' function will be called we infer that the location of the obstacles is unknown but the goal point is known.
"""

# Valuta il vettore di features [h(x_k)] nello stato x_k 
def feature(next_state,goal_points,obs_points,N_feature):
    v = np.array([0.025, 0.025], dtype=np.float32) #vettore delle varianze
    covar = np.diag(v) #matrice di varianze e covarianze
    features = np.zeros(N_feature)
    for i in range(np.size(obs_points,axis=1)):
        features[i+1] = my_logpdf(next_state[:2],obs_points[:2,i],covar) #Modelliamo gli ostacoli come multivariate gaussiane indipendenti (cov=0) centrate nell'ostacolo ipotizzato

    features[0] = (((next_state[0]-goal_points[0])**2 + (next_state[1]-goal_points[1])**2)) #La prima feature è relativa al raggiungimento del goal point

    return features

In [ ]:
%matplotlib inline

# Define the grid of points where to evaluate the feature function
X_axis = np.linspace(X_LIMITS[0]-EPSILON_LIMITS, X_LIMITS[1]+EPSILON_LIMITS, 300)
Y_axis = np.linspace(Y_LIMITS[0]-EPSILON_LIMITS, Y_LIMITS[1]+EPSILON_LIMITS, 200)
X_grid, Y_grid = np.meshgrid(X_axis, Y_axis)

# Calculate the feature function on the grid
features_array = np.array([[feature((x,y),goal_points,obs_points_f,N_feature) for x in X_axis] for y in Y_axis] ) #TODO: speed up this line by vectorizing it


# Plot the feature functions for the obstacles
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,2,1)
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_title('Feature functions (obstacles)', fontdict={'fontsize': 20})
ax.set_xlim(X_LIMITS[0],X_LIMITS[1])
ax.set_ylim(Y_LIMITS[0],Y_LIMITS[1])

ax.scatter(obs_points_f[0,:],obs_points_f[1,:], color='red', zorder=3)


Z = np.sum(features_array[:,:,1:], axis=-1)
colormesh = ax.pcolormesh(X_grid, Y_grid, Z, shading='auto', zorder=1)
ax.contour(X_grid, Y_grid, Z, cmap='coolwarm', levels=5, linewidths=2, linestyles='dashed', alpha=1, zorder=2) #vmin and vmax are used to make the contour lines more visible

# Plot feature function for the goal
ax = fig.add_subplot(1,2,2)
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_title('Feature functions (goal)', fontdict={'fontsize': 20})
ax.set_xlim(X_LIMITS[0],X_LIMITS[1])
ax.set_ylim(Y_LIMITS[0],Y_LIMITS[1])

ax.scatter(goal_points[0,:],goal_points[1,:], color='green', zorder=3)


Z = features_array[:,:,0]
colormesh = ax.pcolormesh(X_grid, Y_grid, Z, shading='auto', zorder=1)
ax.contour(X_grid, Y_grid, Z, cmap='coolwarm', levels=5, linewidths=2, linestyles='dashed', alpha=1, zorder=2) #vmin and vmax are used to make the contour lines more visible


# WP4

In [ ]:
##### WP4: using the previously defined features solve the inverse optimal control problem. 
#          Plot the estimated cost. 
#          Verify that the estimated cost allows the robot to complete the task #####

In [ ]:
%%capture
'''
Solving the convex optimisation problem to learn the cost.
'''
import cvxpy as cp
import numpy as np
import time
M = np.size(X,axis=0) - 1 #size è la lunghezza di tutte le simulazioni concatenate. Concatenare le simulazioni genera 4 termini spuri.
w = cp.Variable((1,N_feature)) #Variabili decisionali. Sono tante quanto il numero di features.
constraints = [w >= 0] #Per hp del problema.
R = np.zeros((99,1)) #??? Mai usato
L = [] #Termini della funzione target da minimizzare (la funzione da minimizzare sum(L))

f_expect = np.zeros((2,20)) #???
feature_sampled = np.zeros((N_feature,M)) #Questo è il valore atteso a sinistra in blu (tranne la moltiplicazione per w che si può portare fuori)
PF = np.zeros((control_space_size,control_space_size,M)) #Questa è la funzione q soprasegnato indicizzata in (u_k, k) dove k è l'indice temporale

for i in range(M): #i è l'i-esimo termine della funzione target da ottimizzare (somma da 1 a M), sto fissando x hat 

    #############################################################################################################################
    features = np.zeros((N_feature,control_space_size,control_space_size)) #Questo è il valore atteso a destra in verde (tranne la moltiplicazione per w che si può portare fuori)
    state = np.array(X[i,:]) #Get the state \hat x_k-1

    x0 = state.reshape(-1,1) #è equivalente alla flatten(), x0 è lo stato iniziale (x0[0],x0[1])
    time_step = 0.033


    pf = np.zeros((control_space_size,control_space_size)) #Initialize pf \overline q(u)

    for j in range(control_space_size):
        for k in range(control_space_size):
            next_state = model_step(state,[U_space_1[j],U_space_2[k]],time_step)
            cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
            f = st.multivariate_normal(next_state.reshape((2,)),cov) #p(x_k|\hat x_k-1, u_k) valutata per \hat x_k-1 e u_k fissati
            #next_sample = f.mean #next_sample = next_state, inutile

            N_samples = 5
            next_samples = f.rvs(N_samples) #5 campioni relativi al prossimo stato. Dimensione 5 campioni x 2 variabili di stato
            feature_sample = np.zeros((N_feature,N_samples)) # 16 feature x 5 campioni

            for m in range(N_samples):
                feature_sample[:,m] = feature(next_samples[m,:],goal_points,obs_points_f,N_feature) #feature_sample è una matrice che tiene sulle colonne le valutazioni delle features sui campioni. Le colonne rorrispondono ai campioni.

            features[:,j,k] = np.mean(feature_sample,axis=1) #la media è fatta su tutti gli elementi della riga [i,;], ovvero per ogni feature i-esima faccio la media su tutti i campioni
            #Alla fine del ciclo features contiene la stima del valore atteso delle features per ogni possibile ingresso (j,k) con \hat x_k-1 fissato
            #quindi ho tutte le stime dei valori attesi a destra in verde nell'equazione. Fatta con la legge dei grandi numeri.
            #Calculate the DKL for each possible input, get corresponding probability
            log_DKL = np.exp(-(-f.entropy())) #Prima parte del valore atteso a destra
            pf[j,k] = log_DKL
    PF[:,:,i] = pf #Prima parte del valore atteso a destra per ogni valore di ingresso e u_k e stato x_k-1

    features = np.reshape(features,(N_feature,control_space_size**2)) # Adesso features ha come primo indice la feature e il secondo va da 0 a 8 e indica l'ingresso di controllo

    f_sampled = model_step(state,U[i+1,:],time_step) # Costuiramo la pdf p(x_k|\hat x_k-1,\hat u_k) che serve a calcolare il valore atteso a sinistra in blu
    cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
    f1 = st.multivariate_normal(f_sampled.reshape((2,)),cov) #p(x_k|\hat x_k-1,\hat u_k)
    next_samples_f1 = f1.rvs(N_samples) #stessa cosa di prima, stima con legge dei grandi numeri del expectation blu a sinistra
    feature_sample_f1 = np.zeros((N_feature,N_samples))
    for n in range(N_samples):
        feature_sample_f1[:,n] = feature(next_samples_f1[n,:],goal_points,obs_points_f,N_feature)

    feature_sampled[:,i] = np.mean(feature_sample_f1,axis=1)  # features_sampled è la stessa cosa di features fatta sul valore atteso blu. nota che u è fissata qunidi non ho come indici j e k. Ho i come indice ed è inutile.

    # Task: solve, using cvx the convex optimization problem we saw in class. To do so:
    # (i) prepare each individual term of the summation, say l;
    # (ii) sum all the elements to define the cost function
    # (iii) solve the problem
    PF2 = np.reshape(PF,(-1,M))
    # log_sum_exp prende in ingresso il vettore colonna degli elmenti da sommare
    l = -(w @ feature_sampled[:,i]) + cp.log_sum_exp(cp.reshape(w@features[:,:],(control_space_size**2,)) + cp.log(PF2[:,i])) #(i)
    L.append(l) #(ii)

objective = cp.Minimize(cp.sum(L)) #(iii)

prob = cp.Problem(objective)

result = prob.solve(verbose = True)

print("status:", prob.status)
print("optimal value", prob.value)

In [ ]:
# Show the values: critically discuss if these weights make sense
weights = w.value

print('weights:',weights)

In [ ]:
# Check the status of the optimization problem: did the optimization go well? Si
print("status:", prob.status)
print("optimal value", prob.value)

### Plot the estimated cost function

In [ ]:
# Reformatting the original cost map (just for checking and plotting purposes)

import numpy as np
import scipy.stats as st
import pandas as pd

goal_points = np.array(np.mat('-1.4; -0.8; 0'))

#obs_points = np.array(np.mat('0 0 0 0 0 0;0 0.2 0.4 0.6 0.8 -0.8;0 0 0 0 0 0'))
obs_points = np.array(np.mat('0 0 0 0 0;0.2 0.4 0.6 0.8 -0.8;0 0 0 0 0'))

def state_cost(state,goal_points,obs_points):
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)

    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum
    return(cost)


Cost_Map = np.zeros((300,200))
X_axis = np.linspace(-1.5,1.5,300)
Y_axis = np.linspace(-1,1,200)

for i in range(200):
    for j in range(300):

        state = np.array([X_axis[j],Y_axis[i]])
        Cost_Map[j,i] = state_cost(state,goal_points,obs_points)

# we delete this row because we understand that it is not used
#Coat_Map = pd.DataFrame(Cost_Map,index=list(X_axis),columns=Y_axis)

In [ ]:
# Computing the reconstructed cost map

import numpy as np
import scipy.stats as st

#goal_points = np.array(np.mat('-1.4; -0.8; 0'))

def state_cost_estimated(state,goal_points,obs_points,weights):
    v = np.array([0.025, 0.025], dtype=np.float32)
    covar = np.diag(v)

    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += -weights[:,i+1]*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = -weights[:,0]*((((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2))) + gauss_sum
    return(cost)


Cost_Map = np.zeros((300,200))
X_axis = np.linspace(-1.5,1.5,300)
Y_axis = np.linspace(-1,1,200)

for i in range(200):
    for j in range(300):

        state = np.array ([X_axis[j],Y_axis[i]])
        Cost_Map[j,i] = state_cost_estimated(state,goal_points,obs_points_f,weights)

Heatmap of the estimated cost function

In [ ]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.colors import CenteredNorm
# Transpose the data array to rotate the heatmap
data_rotated = np.transpose(Cost_Map) 
 
fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot()
ax.set_xlabel('X [m]', labelpad=20)
ax.set_ylabel('Y [m]', labelpad=20)
ax.set_title('Estimated cost function', fontdict={'fontsize': 20})

# Plot the heatmap of the cost function
pcolormesh = ax.pcolormesh(X_axis, Y_axis, data_rotated, cmap='coolwarm', shading='auto', zorder=1)

# Plot the level curves of the cost function
contour = ax.contour(X_axis, Y_axis, data_rotated, cmap='coolwarm', shading='linear', levels=7, linewidths=2, linestyles='dashed', zorder=2, alpha=1, vmin=40, vmax=100) #vmin and vmax are used to make the contour lines more visible

# Add a colorbar
fig.colorbar(pcolormesh, ax=ax)

# Create a rectangle
rectangle = Rectangle((X_LIMITS[0], Y_LIMITS[0]), X_LIMITS[1]-X_LIMITS[0], Y_LIMITS[1]-Y_LIMITS[0], fill=False, color="black", linewidth=1, zorder=3)
# Add the rectangle to the plot
ax.add_patch(rectangle)
 
plt.scatter(obs_points_f[0,:],obs_points_f[1,:], color='red', zorder=3)
plt.scatter(goal_points[0,:],goal_points[1,:], color='green', zorder=3)

plt.show()




3D plot of the estimated cost function

In [ ]:
%matplotlib tk
# Create a 3D plot
fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(projection='3d')
ax.set_xlabel('X [m]', labelpad=10)
ax.set_ylabel('Y [m]', labelpad=10)
ax.set_zlabel('Cost')
ax.set_title('Estimated Cost Function', fontdict={'fontsize': 20})

surf = ax.plot_surface(X_grid, Y_grid, data_rotated, cmap=cm.coolwarm, linewidth=0, antialiased=False, ccount=100, rcount=100) #rcount and ccount are used to make the surface smoother
fig.colorbar(surf, shrink=0.5)
plt.show()



### Simulation with the estimated cost function

In [ ]:
#TODO: delete interpolated cost

We calculate an interpolation of the cost function to speed up the computation of the policy ---- da togliere prima  di consegnare

In [ ]:
from scipy import interpolate

X_axis_interpolation = np.linspace(X_LIMITS[0]-0.5, X_LIMITS[1]+0.5, 600)
Y_axis_interpolation = np.linspace(Y_LIMITS[0]-0.5, Y_LIMITS[1]+0.5, 400)
X_grid_interpolation, Y_grid_interpolation = np.meshgrid(X_axis_interpolation, Y_axis_interpolation)
Z = np.zeros((600,400))
for i in range(600):
    for j in range(400):
        Z[i,j] = state_cost_estimated(np.array([X_axis_interpolation[i],Y_axis_interpolation[j]]),goal_points,obs_points_f,weights)

# interpolated_costs = griddata((X_interpolation.flatten(),Y_interpolation.flatten()), Z.flatten(), (X_interpolation.flatten(), Y_interpolation.flatten()), method='cubic').reshape(dense_X.shape)

# Cost_Map_2 = np.zeros((600,400))
# X_axis_2 = np.linspace(-3.0,3.0,600)
# Y_axis_2 = np.linspace(-2,2,400)

# for i in range(400):
#     for j in range(600):
#         state = np.array([X_axis_2[j],Y_axis_2[i]])
#         Cost_Map_2[j,i] = state_cost_estimated(state,goal_points,obs_points_f,weights)

# interpolated_cost = interpolate.RegularGridInterpolator((X_axis_2, Y_axis_2), Cost_Map_2)
# interpolated_cost([1.49,1.0])

interpolated_cost = interpolate.LinearNDInterpolator(np.stack((X_grid_interpolation.flatten(),Y_grid_interpolation.flatten()),axis=1),Z.T.flatten())

Plot the interpolated cost function

In [ ]:
#Plot the heatmap of the interpolated cost function
fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot()
ax.set_xlabel('X [m]', labelpad=20)
ax.set_ylabel('Y [m]', labelpad=20)
ax.set_title('Interpolated cost function', fontdict={'fontsize': 20})
TEST = np.zeros((600,400))

for i in range(600):
    for j in range(400):
        TEST[i,j] = interpolated_cost(X_axis_interpolation[i], Y_axis_interpolation[j])
pcolormesh = ax.pcolormesh(X_grid_interpolation, Y_grid_interpolation, TEST.T, cmap='coolwarm',  zorder=1)


In [ ]:
#Task: re-define the function Control_step so that it now uses the estimated cost
def Control_step(state,U_space_1,U_space_2,goal_points,obs_points):
    ###
    # Perform a control step given the fact that the target pf is uniform.
    # The function first gets the target pf (uniform) and then applies the control solution we saw in class
    
    target_pf = 1/control_space_size**2 # Uniform pf q(u_k|x_k-1)
    time_step = 0.033 # The Robotarium time-step

    pf = np.zeros((control_space_size,control_space_size))
    for i in range(control_space_size):
        for j in range(control_space_size):
            next_state = model_step(state,[U_space_1[i],U_space_2[j]],time_step)
            cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
            f = st.multivariate_normal(next_state.reshape((2,)),cov)
            N_samples = 20
            next_sample = f.rvs(N_samples) 

            # Task: what do the next three lines do?
            # Calcola una stima del valore atteso del costo nello stato successivo rispetto alla funzione f(x_k|x_k-1,u_k) usando 20 campioni
            cost=0
            for k in range(N_samples):
                #cost += state_cost_estimated(next_sample[k,:],goal_points,obs_points_f,weights)/N_samples
                cost += interpolated_cost(next_sample[k,:])/N_samples
            log_DKL = np.exp(f.entropy()-cost).item() #item is used to convert the result to a scalar
            pf[i,j] = log_DKL 
    # Task: obtain the normalizer for the policy, call it S2
    S2 = np.sum(pf)
    # Task: obtain the normalized pf (call the variable pf)
    pf = np.divide(pf,S2)

    # Adesso abbiamo p(u_k|x_{k-1}) e facciamo il sampling della prossima azione di controllo
    # This is a trick to properly sample from the multi-dimensional pf
    flat = pf.flatten()
    sample_index = np.random.choice(a=flat.size, p=flat)
    # Take this index and adjust it so it matches the original array
    adjusted_index = np.unravel_index(sample_index, pf.shape) #Indice dell'azione

    #Formatta l'azione come vettore colonna
    action = np.reshape(np.array([U_space_1[adjusted_index[0]],U_space_2[adjusted_index[1]]]),(2,1))
    return(action)

In [ ]:
%matplotlib tk
# Instantiate Robotarium object (start the robots from different initial conditions than the 4 experiments above)
N = 1

MAX_ITERATIONS = 5000
initial_conditions = [np.array(np.mat('-1.4;0.9; 0')),np.array(np.mat('1;0.9; 0')),np.array(np.mat('1;-0.25; 0'))]
N_experiment = 3
# X_si is going to be two-dimensional state history
X_Si = [0]*N_experiment
# D_Xi is going to be two-dimensional inputs history
D_Xi = [0]*N_experiment

# This first for loop creates the initial conditions

for I in range(N_experiment):

    X_si = []
    D_xi = []

    r = robotarium.Robotarium(number_of_robots=N, show_figure=True, initial_conditions=np.copy(initial_conditions[I]), sim_in_real_time=False)

    si_to_uni_dyn = create_si_to_uni_dynamics_with_backwards_motion()

    x = r.get_poses()
    x_si = uni_to_si_states(x)

    CM = np.random.rand(N+10,3) 
    goal_marker_size_m = 0.15
    obs_marker_size_m = 0.15
    marker_size_goal = determine_marker_size(r,goal_marker_size_m)
    marker_size_obs = determine_marker_size(r,obs_marker_size_m)
    font_size = determine_font_size(r,0.1)
    line_width = 5

    goal_caption = ['G{0}'.format(ii) for ii in range(goal_points.shape[1])]
    goal_points_text = [r.axes.text(goal_points[0,ii], goal_points[1,ii], goal_caption[ii], fontsize=font_size, color='k',fontweight='bold',horizontalalignment='center',verticalalignment='center',zorder=-2)
    for ii in range(goal_points.shape[1])]
    goal_markers = [r.axes.scatter(goal_points[0,ii], goal_points[1,ii], s=marker_size_goal, marker='s', facecolors='none',edgecolors=CM[ii,:],linewidth=line_width,zorder=-2)
    for ii in range(goal_points.shape[1])]

    obs_caption = ['OBS{0}'.format(ii) for ii in range(obs_points.shape[1])]
    obs_points_text = [r.axes.text(obs_points[0,ii], obs_points[1,ii], obs_caption[ii], fontsize=font_size, color='k',fontweight='bold',horizontalalignment='center',verticalalignment='center',zorder=-2)
    for ii in range(obs_points.shape[1])]
    obs_markers = [r.axes.scatter(obs_points[0,ii], obs_points[1,ii], s=marker_size_obs, marker='s', facecolors='none',edgecolors=CM[ii+1,:],linewidth=line_width,zorder=-2)
    for ii in range(obs_points.shape[1])]

    r.step()
    i = 0
    # While the robot is away from the objective ...
    while (np.size(at_pose(np.vstack((x_si,x[2,:])), goal_points, position_error=0.15,rotation_error=100)) != N and i < MAX_ITERATIONS ):

        # Get poses of agents
        x = r.get_poses()
        x_si = uni_to_si_states(x)

        #Add to the dataset
        X_si.append(x_si)

        # The lines below define the pdf of the robot 
        cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
        x_pdf = st.multivariate_normal(x_si.reshape((2,)),cov)
        x_sample = x_pdf.rvs() #Noisy state

        # This is about plotting
        for j in range(goal_points.shape[1]):
            goal_markers[j].set_sizes([determine_marker_size(r, goal_marker_size_m)])

        for j in range(obs_points.shape[1]):
            obs_markers[j].set_sizes([determine_marker_size(r, obs_marker_size_m)])

        # Task: compute the action from the policy. Call the variable dxi: 
        # this is the action sampled from the optimal solution to the control problem
        dxi = Control_step(x_sample, U_space_1, U_space_2, goal_points, obs_points)
        D_xi.append(dxi)

        # Transform single integrator velocity commands to unicycle inputs (low level controller)
        dxu = si_to_uni_dyn(dxi, x)

        # Set the velocities inputs
        r.set_velocities(np.arange(N), dxu)
        # Iterate the simulation
        r.step()
        
        i+=1
      

    D_Xi[I] = D_xi
    X_Si[I] = X_si

    r.call_at_scripts_end()
    plt.close()

In [ ]:
XX = X_Si
UU = D_Xi

In [ ]:
X = []
X_plot = []
U = []
U_plot = []

for i in range(len(XX)):
    X.append(np.array(XX[i]))
    X_plot.append(np.array(XX[i]))

X = np.concatenate(X, axis=0)
X = np.reshape(X, (-1, 2))

U = []
for i in range(len(UU)):
    U.append(np.array(UU[i]))
    U_plot.append(np.array(UU[i]))

U = np.concatenate(U, axis=0)
U = np.reshape(U, (-1, 2))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np
%matplotlib inline
#Task: plot trajectories with different colors
plt.figure(figsize=(15,10))

for I in range(N_experiment):
    trajectory = np.array(X_Si[I])
    plt.plot(trajectory[:,0],trajectory[:,1])

# Add a star to the starting point of each trajectory
for I in range(N_experiment):
    trajectory = np.array(X_Si[I])
    plt.scatter(trajectory[0,0], trajectory[0,1], marker='*', s=200, color='black')

# Add a legend to the plot
legend = []
for i in range(N_experiment):
    legend = legend + ['Experiment {}'.format(i+1)]
    plt.legend(legend)


#Draw goal point
square = plt.Rectangle((float(goal_points[0][0])-0.15,float(goal_points[1][0])-0.15), 0.3, 0.3, fc='green',ec="black") # Obiettivo da raggiungere
plt.gca().add_patch(square)

plt.scatter(obs_points_f[0,:],obs_points_f[1,:])

#Draw obstacles
square1 = plt.Rectangle((-1.6,-1), 0.4, 0.4, fc='green',ec="black")
square3 = plt.Rectangle((-0.2,-1), 0.4, 0.4, fc='red',ec="black")
square2 = plt.Rectangle((-0.2,0), 0.4, 0.8, fc='red',ec="black")
plt.gca().add_patch(square2)
plt.gca().add_patch(square1)
plt.gca().add_patch(square3)
plt.ylim(-1,1)
plt.xlim(-1.5,1.5)
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.show()

In [ ]:
# Task: plot the feature points on the robotarium grid with corresponding weights
%matplotlib inline

import matplotlib.pyplot as plt
print(weights)
fig, ax = plt.subplots()
ax.scatter(obs_points_f[0,:],obs_points_f[1,:])
for i, w in enumerate(weights[0,1:]):
    ax.annotate(round(w,2), (obs_points_f[0,i] + 0.02, obs_points_f[1,i] + 0.02))

# Set x-axis limits
ax.set_xlim([-1.5, 1.5])

# Set y-axis limits
ax.set_ylim([-1, 1])

In [ ]:
# Comment the results you observe in the figure generated by the above cell

# WP5

## Refactoring the code

In [ ]:
#REDEFINING THE OLD FUNCTION:

import rps.robotarium as robotarium
from rps.utilities.transformations import *
from rps.utilities.barrier_certificates import *
from rps.utilities.misc import *
from rps.utilities.controllers import *
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

import scipy.stats as st
import time
import numpy as np
# Defining the input axes
# Note: the boundaries for the control actions are the actuation constraints imposed by the physics of the robots

control_space_size = 3  # Three possible inputs for each control axis

U_space_1 = np.array(np.linspace((-0.5),(0.5),control_space_size)) # Control space for the first axis
U_space_2 = np.array(np.linspace((-0.5),(0.5),control_space_size)) # Control space for the second axis
time_step = 0.033 # Robotarium time-step (from the documentation)


# This function performs a "model" step using the documented dynamics
# Note: from the viewpoint of the controller the dynamics is not necesarily known
def model_step(x,velocities,time_step):
    """This function calculates the next pose of the robots based on the current pose, the velocities and the simulation time-step.
    Args:
        x : The current position of the robot. 2x1 column vector .
        velocities : The velocities of the robot along the two control axes. 2x1 column vector. 
        time_step : simulation time-step
    Returns:
        The next pose of the robot as a 2x1 column vector.
    """
    poses = np.zeros((2,1))
    # Update pose of the robots
    poses[0] = x[0] + time_step*velocities[0]
    poses[1] = x[1] + time_step*velocities[1]
    return(poses)

#Get the value of a Gaussian pf at a given point x, with mean u and covariance covar
def my_logpdf(x, u, covar):
    """This function calculates the value of a multivariate Gaussian pdf at a given point x, with mean u and covariance covar

    Args:
        x : The point at which the pdf is evaluated. Nx1 column vector.
        u : The mean vector of the Gaussian distribution. Nx1 column vector.
        covar : The covariance matrix of the Gaussian distribution. NxN matrix.

    Returns:
        The value of the Gaussian pdf at x.
    """
    k = len(x)  # dimension
    a = np.transpose(x - u)
    b = np.linalg.inv(covar)
    c = x - u
    d = np.matmul(a, b)
    e = np.matmul(d, c)
    numer = np.exp(-0.5 * e)
    f = (2 * np.pi)**k
    g = np.linalg.det(covar)
    denom = np.sqrt(f * g)
    pdf = numer / denom
    return pdf


##### WP0: formalize the control problem #####

# Task: reverse engineer the cost function used by the robots. What is the problem formulation? 
#      Is the one below a good cost for the task? Create a heatmap to visualize the cost 

def state_cost(state,goal_points,obs_points):
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum + 10*(np.exp(-0.5*((state[0]-(-1.5))/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/0.02)**2)/(0.02*np.sqrt(2*np.pi)))
    return(cost)

##### WP1: fill-in the code for the function below.
#         The function needs to return the optimal action sampled from the optimal policy.
#         The action is used in the simulation loop #####

def Control_step(state,U_space_1,U_space_2,goal_points,obs_points, cost_func):
    ###
    # Perform a control step given the fact that the target pf is uniform.
    # The function first gets the target pf (uniform) and then applies the control solution we saw in class
    
    target_pf = 1/control_space_size**2 # Uniform pf q(u_k|x_k-1)
    time_step = 0.033 # The Robotarium time-step

    pf = np.zeros((control_space_size,control_space_size)) #Initialize pf
    for i in range(control_space_size):
        for j in range(control_space_size):
            # Task: what do the next three lines do?
            """
            The next three lines calculate the probability function of the next state x_k given the current state x_{k-1} and the action u_k.
            The probability function is a multivariate Gaussian distribution with mean vector x_k = x_{k-1} + u_k * time_step and covariance matrix cov = [[0.001, 0.0002], [0.0002, 0.001]].
            """
            next_state = model_step(state,[U_space_1[i],U_space_2[j]],time_step)
            cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
            f = st.multivariate_normal(next_state.reshape((2,)),cov) #p(x_k|u_k,x_k-1)
            # Task: what do the next two lines do?
            """
            The next two lines extract 20 samples from the probability function p(x_k|u_k,x_k-1) calculated before.
            """
            N_samples = 20
            next_sample = f.rvs(N_samples) 

            # Task: what do the next three lines do?
            """
            The next three lines calculate an expectation of the cost function as a mean of the cost function evaluated on the 20 samples extracted before.
            This is done to estimate the value of the cost function in the next state.
            According to the law of large numbers, the more samples we extract, the more accurate the estimation will be.
            """
            # Calcola una stima del valore atteso del costo nello stato successivo rispetto alla funzione f(x_k|x_k-1,u_k) usando 20 campioni
            cost=0
            #lnp = 0
            for k in range(N_samples):
                cost += cost_func(next_sample[k,:],goal_points,obs_points)/N_samples
                #lnp += np.log(f.pdf(next_sample[k,:]))/N_samples
            # Task: write here a line of code, defining the variable log_DKL that contains the exponential in the policy
            """
            Here we exploit the fact that the entropy of a pf f is defined as the expectation of the negative logarithm of f.
            """
            log_DKL = np.exp(f.entropy()-cost).item() #item() function is called to cast the ndarray to a scalar which avoids a warning
            pf[i,j] = log_DKL #Calculate the DKL for each possible input, get corresponding probability
    # Task: obtain the normalizer for the policy, call it S2
    """
    The normalizer for the policy is the sum of the pf calculated on each possible input.
    """
    S2 = np.sum(pf)
    # Task: obtain the normalized pf (call the variable pf)
    """
    To normalize the pf, we divide it by the normalizer S2.
    """
    pf = np.divide(pf,S2)

    # Adesso abbiamo p(u_k|x_{k-1}) e facciamo il sampling della prossima azione di controllo
    # This is a trick to properly sample from the multi-dimensional pf
    flat = pf.flatten()
    sample_index = np.random.choice(a=flat.size, p=flat) # We sample an action from the flattened pf
    # Take this index and adjust it so it matches the original array
    adjusted_index = np.unravel_index(sample_index, pf.shape) # This function converts the index of the flattened pf back into a tuple of two indices for the original action space

    action = np.reshape(np.array([U_space_1[adjusted_index[0]],U_space_2[adjusted_index[1]]]),(2,1)) # We extract the control actions on the two control axes and we reshape them into a 2x1 column vector

    return(action)

def robotarium_simulation(initial_conditions = [np.array(np.mat('1.4;0.9; 0')),np.array(np.mat('0.2;0.9; 0')),np.array(np.mat('1.2;-0.5; 0')),np.array(np.mat('-1;0.9; 0'))], 
                          max_iteration=2500, show_simulation=True, cost_function=state_cost):
    %matplotlib tk
    # Instantiate Robotarium object
    N = 1 #Amount of robots per simulation
    MAX_ITERATIONS = max_iteration
    SHOW_SIMULATIONS= show_simulation

    N_experiment = len(initial_conditions)
    # X_si is going to be two-dimensional state history
    X_Si = [0]*N_experiment
    # D_Xi is going to be two-dimensional inputs history
    D_Xi = [0]*N_experiment

    # This first for loop creates the initial conditions
    for I in range(N_experiment):

        X_si = []
        D_xi = []

        r = robotarium.Robotarium(number_of_robots=N, show_figure=SHOW_SIMULATIONS, initial_conditions=np.copy(initial_conditions[I]), sim_in_real_time=False)

        # Create mapping from the control inputs to the actual velocity commands to the unicycle
        # Note: this is a very practical situation (robots often provide transformation functions to low level commands)
        si_to_uni_dyn = create_si_to_uni_dynamics_with_backwards_motion() #Converts single integrator inputs to unicycle inputs (low-level controller)
        _, uni_to_si_states = create_si_to_uni_mapping()
        
        # define x initially
        x = r.get_poses()
        x_si = uni_to_si_states(x)

        # Plotting Parameters
        CM = np.random.rand(N+10,3) # Random Colors
        goal_marker_size_m = 0.15
        obs_marker_size_m = 0.15
        marker_size_goal = determine_marker_size(r,goal_marker_size_m)
        marker_size_obs = determine_marker_size(r,obs_marker_size_m)
        font_size = determine_font_size(r,0.1)
        line_width = 5

        # Create Goal Point Markers
        #Text with goal identification
        goal_caption = ['G{0}'.format(ii) for ii in range(goal_points.shape[1])]
        #Plot text for caption
        goal_points_text = [r.axes.text(goal_points[0,ii], goal_points[1,ii], goal_caption[ii], fontsize=font_size, color='k',fontweight='bold',horizontalalignment='center',verticalalignment='center',zorder=-2)
        for ii in range(goal_points.shape[1])]
        goal_markers = [r.axes.scatter(goal_points[0,ii], goal_points[1,ii], s=marker_size_goal, marker='s', facecolors='none',edgecolors=CM[ii,:],linewidth=line_width,zorder=-2)
        for ii in range(goal_points.shape[1])]

        #Text with goal identification
        obs_caption = ['OBS{0}'.format(ii) for ii in range(obs_points.shape[1])]
        #Plot text for caption
        obs_points_text = [r.axes.text(obs_points[0,ii], obs_points[1,ii], obs_caption[ii], fontsize=font_size, color='k',fontweight='bold',horizontalalignment='center',verticalalignment='center',zorder=-2)
        for ii in range(obs_points.shape[1])]
        obs_markers = [r.axes.scatter(obs_points[0,ii], obs_points[1,ii], s=marker_size_obs, marker='s', facecolors='none',edgecolors=CM[ii+1,:],linewidth=line_width,zorder=-2)
        for ii in range(obs_points.shape[1])]

        r.step()

        i = 0
        # While the robot is away from the objective ...
        while (np.size(at_pose(np.vstack((x_si,x[2,:])), goal_points, position_error=0.15,rotation_error=100)) != N and i < MAX_ITERATIONS ):

            # Get poses of agents
            x = r.get_poses()
            x_si = uni_to_si_states(x)

            #Add to the dataset
            X_si.append(x_si)

            # The lines below define the pdf of the robot 
            cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
            x_pdf = st.multivariate_normal(x_si.reshape((2,)),cov)
            x_sample = x_pdf.rvs() #Noisy state

            # This is about plotting
            for j in range(goal_points.shape[1]):
                goal_markers[j].set_sizes([determine_marker_size(r, goal_marker_size_m)])

            for j in range(obs_points.shape[1]):
                obs_markers[j].set_sizes([determine_marker_size(r, obs_marker_size_m)])

            # Task: compute the action from the policy. Call the variable dxi: 
            # this is the action sampled from the optimal solution to the control problem
            dxi = Control_step(x_sample, U_space_1, U_space_2, goal_points, obs_points, cost_function)
            D_xi.append(dxi)

            # Transform single integrator velocity commands to unicycle inputs (low level controller)
            dxu = si_to_uni_dyn(dxi, x)

            # Set the velocities inputs
            r.set_velocities(np.arange(N), dxu)
            # Iterate the simulation
            r.step()
            
            i += 1

        if i >= MAX_ITERATIONS:
            print("SIMULATION {} FAILED!".format(I))
        
        D_Xi[I] = D_xi
        X_Si[I] = X_si

        #Call at end of script to print debug information and for your script to run on the Robotarium server properly
        r.call_at_scripts_end()
        plt.close()
        
    return X_Si, D_Xi

## Alternative cost functions

In [ ]:
# Altertive cost functions
def state_cost_without_borders(state,goal_points,obs_points):
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum
    return(cost)

def state_cost_with_additional_term(state,goal_points,obs_points):
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)
    state_cost = ((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) -1/((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2 +0.1)
    cost = 30*state_cost + gauss_sum + 10*(np.exp(-0.5*((state[0]-(-1.5))/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/0.02)**2)/(0.02*np.sqrt(2*np.pi)))
    return(cost)

def indicator_function(X,Y):
    """ x > o < di una certa quantità e y > o < di una certa quantità """
    condition = np.logical_or(np.logical_or(X < -1.5, X > 1.5), np.logical_or(Y < -1.0, Y > 1.0))
    inf = 500
    result = np.where(condition, inf, 0)
    return result



def my_cilinder(x,u):
    r = 0.4
    k = 1000
    height = 100
    
    temp = height*(-1/(1+np.exp(-k*((x[0]-u[0])**2+(x[1]-u[1])**2-(r**2)))) + 1)
    return temp

def state_cost_cone_for_goal_point_cilinder_for_obstacles(state,goal_points,obs_points):

    value = 0.05
    a = 0.5
    b = 0.5

    v = np.array([value, value], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        
        gauss_sum += my_cilinder(state[:2],obs_points[:2,i])

    
    if np.abs(state[0]-goal_points[0]) < 2 or np.abs(state[1]-goal_points[1]) < 2:
        goal_point_cost = np.sqrt((state[0]-goal_points[0])**2/(a**2) + (state[1]-goal_points[1])**2/(b**2))
    else:
        goal_point_cost = (state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2
    
    cost = 30*goal_point_cost + gauss_sum + 10*(np.exp(-0.5*((state[0]-(-1.5))/value)**2)/(value*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/value)**2)/(value*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/value)**2)/(value*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/value)**2)/(value*np.sqrt(2*np.pi)))
    return(cost)

def state_cost_higher_goal_weight(state,goal_points,obs_points):
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 60*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum + 10*(np.exp(-0.5*((state[0]-(-1.5))/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/0.02)**2)/(0.02*np.sqrt(2*np.pi)))
    return(cost)

def state_cost_bigger_border(state,goal_points,obs_points):
    displacement = 0.20
    dev_std_border=0.08
    multiplier_factor=dev_std_border/0.02
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum + 10*(
                multiplier_factor* np.exp(-0.5*((state[0]-(-1.5 -displacement ))/dev_std_border)**2)/(dev_std_border*np.sqrt(2*np.pi))
                + multiplier_factor* np.exp(-0.5*((state[0]-1.5 - displacement)/dev_std_border)**2)/(dev_std_border*np.sqrt(2*np.pi)) 
                + multiplier_factor* np.exp(-0.5*((state[1]-1.0 -displacement)/dev_std_border)**2)/(dev_std_border*np.sqrt(2*np.pi))
                + multiplier_factor* np.exp(-0.5*((state[1]-(-1.0 -displacement))/dev_std_border)**2)/(dev_std_border*np.sqrt(2*np.pi)))
    return(cost)

dropwave = lambda X,Y,f,s,p,o,k,A, obstacle: A*(o + np.cos(f*np.sqrt(((X-obstacle[0])/k)**2+((Y-obstacle[1])/k)**2))/((s)*((X-obstacle[0])**2+(Y-obstacle[1])**2)+p))
dropwave_attenuation = lambda X,Y,f,s,p,o,k,A,goal,obstacle,alpha,beta: dropwave(X,Y,f,s,p,o,k,A, obstacle) * np.exp(alpha*np.sqrt((X-goal[0])**2+(Y-goal[1])**2)) * np.exp(-beta*np.sqrt((X-obstacle[0])**2+(Y-obstacle[1])**2))

def experimental(state, goal_points, obs_points, parameters=None):
    if parameters is None:
        f =  57.67374
        s =  100.0001
        p =  30.000100000000003
        o =  0.0
        k =  4.185
        A =  21.035
        alpha =  4.54734
        beta =  5.917910000000001
    else:
        f = parameters['f']
        s = parameters['s']
        p = parameters['p']
        o = parameters['o']
        k = parameters['k']
        A = parameters['A']
        alpha = parameters['alpha']
        beta = parameters['beta']
    
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar) + dropwave_attenuation(state[0],state[1],f,s,p,o,k,A,goal_points,obs_points[:2,i],alpha,beta)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum + 10*(np.exp(-0.5*((state[0]-(-1.5))/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/0.02)**2)/(0.02*np.sqrt(2*np.pi)))
    return(cost)

exp_distance = lambda X,Y,Lambda: np.exp(Lambda*np.sqrt(X**2+Y**2))
single_wave = lambda X,Y,goal, A, k,omega,Lambda, alpha: A*(exp_distance(X,Y,Lambda)/k**2+np.sin(omega*exp_distance(X,Y,Lambda)/k**2))*np.exp(-(exp_distance(X,Y,Lambda)/k**2)**2) * np.exp(alpha*np.sqrt((X-goal[0])**2+(Y-goal[1])**2))

euclidean_distance = lambda X,Y,O=np.array([0,0]): np.sqrt((X-O[0])**2+(Y-O[1])**2)
manhattan_distance = lambda X,Y,O=np.array([0,0]): np.maximum(np.abs(X-O[0]),np.abs(Y-O[1]))
vector_module = lambda X,Y: np.sqrt(X**2+Y**2)
gaussian = lambda X, mu, sigma: np.exp(-0.5*((X-mu)/sigma)**2)/(sigma*np.sqrt(2*np.pi))
theta = lambda X,Y,G,O: np.arccos(((X-G[0])*(O[0]-G[0]) + (Y-G[1])*(O[1]-G[1]))/(vector_module(X-G[0],Y-G[1])+0.00001)/(vector_module(O[0]-G[0],O[1]-G[1])+0.00001))
#decay = lambda X,Y,G,O,l: np.exp(-l*(theta(X,Y,G,O)/np.pi)**2)
decay = lambda X,Y,G,O,l: theta(X,Y,G,O)<l*np.pi/100
circular_barrier = lambda X,Y,center,r,sigma,A,obstacle,l: A*gaussian(euclidean_distance(X,Y,center),r+manhattan_distance(obstacle[0],obstacle[1],center),sigma)*decay(X,Y,center,obstacle,l)

def single_wave_cost(state, goal_points, obs_points, parameters=None):
    if parameters is None:
        r = 0.266390
        sigma = 0.226100
        A = 27.200140
        l = 0.000000
    else:
        r = parameters['r']
        sigma = parameters['sigma']
        A = parameters['A']
        l= parameters['l']
    
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += circular_barrier(state[0],state[1],goal_points,r,sigma,A,obs_points[:2,i],l) #+ 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum + 10*(np.exp(-0.5*((state[0]-(-1.5))/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/0.02)**2)/(0.02*np.sqrt(2*np.pi)))
    return(cost)

experimental_cost_generator = lambda params: lambda state, goal_points, obs_points: experimental(state, goal_points, obs_points, params)
single_wave_cost_generator = lambda params: lambda state, goal_points, obs_points: single_wave_cost(state, goal_points, obs_points, params)


def my_cost(state, goal_points, obs_points,params):

    rho = params['rho']
    var_x = params['var_x']
    var_y = params['var_y']
    A = params['A']

    barrier_variance = 0.05
    cov = np.array([[var_x, rho*np.sqrt(var_x)*np.sqrt(var_y)], [rho*np.sqrt(var_x)*np.sqrt(var_y), var_y]])

    goal_cost = (state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2 -1/np.sqrt(((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2 +0.1))

    obstacle_cost = 0
    for i in range(np.size(obs_points,axis=1)):
        obstacle_cost += A*my_logpdf(state[:2],obs_points[:2,i],cov) + 5*my_logpdf(state[:2],obs_points[:2,i],np.array([[0.008, 0], [0, 0.008]]))
    


    barrier_cost = 10*(np.exp(-0.5*((state[0]-(-1.5))/barrier_variance)**2)/(barrier_variance*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/barrier_variance)**2)/(barrier_variance*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/barrier_variance)**2)/(barrier_variance*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/barrier_variance)**2)/(barrier_variance*np.sqrt(2*np.pi))) + indicator_function(state[0],state[1])



    return (30*goal_cost + obstacle_cost + barrier_cost)

def my_cost_generator(params):
    return lambda state, goal_points, obs_points: my_cost(state, goal_points, obs_points, params)

In [ ]:
cost_parameters = {
    'baseline':{
        'f': 57.67374,
        's': 100.0001,
        'p': 30.000100000000003,
        'o': 0.0,
        'k': 4.185,
        'A': 21.035,
        'alpha': 4.54734,
        'beta': 5.917910000000001,
    },
    'test':{
        'r': 0.266390,
        'sigma': 0.226100,
        'A': 27.200140,
    },
    'test2':{
        'r': 0.564660,
        'sigma': 0.089060,
        'A': 4.627710,
    },
    'test3':{
        'r': 0.161600,
        'sigma': 0.140310,
        'A': 38.486350,
        'l': 4.645100,
    },
    'bivariate':{
        'rho': 0.000000,
        'var_x': 0.140000,
        'var_y': 0.140000,
        'A': 30.000000
    }
}

In [ ]:
INITIAL_CONDITIONS_BASELINE = [np.array(np.mat('1.4;0.9; 0')),np.array(np.mat('0.2;0.9; 0')),np.array(np.mat('1.2;-0.5; 0')),np.array(np.mat('-1;0.9; 0'))]
GOAL_POINTS_BASELINE = np.array(np.mat('-1.4; -0.8; 0'))
OBS_POINTS_BASELINE = np.array(np.mat('0 0 0 0 0;0.2 0.4 0.6 0.8 -0.8;0 0 0 0 0'))
COST_FUNCTION_BASELINE = state_cost

SCENARIOS = {
    'baseline': {
        'initial_conditions': INITIAL_CONDITIONS_BASELINE,
        'goal_points': GOAL_POINTS_BASELINE,
        'obs_points': OBS_POINTS_BASELINE
    },
    'six-robots-towards-walls': {
        'initial_conditions': [
            np.array(np.mat('1.4;0.9; 0')),
            np.array(np.mat('0.3;0.9; 1.57')),
            np.array(np.mat('1.2;-0.5; 0')),
            np.array(np.mat('-0.3;0.9; 1.57')),
            np.array(np.mat('-1.40;0.625; 3.14')),
            np.array(np.mat('0.5;-0.9; -1.57'))],
        'goal_points': GOAL_POINTS_BASELINE,
        'obs_points': OBS_POINTS_BASELINE
    },
    'obstacles-ahead-goal': {
        'initial_conditions': [np.array(np.mat('1.4;0.9; 0')),
                               np.array(np.mat('0.2;0.9; 0')),
                               np.array(np.mat('1.2;-0.8; 0')),
                               np.array(np.mat('-1;0.9; 0')),
                               np.array(np.mat('-1;-0.75; 0')),
                               np.array(np.mat('-1.2;-0.15; 0')),
                               np.array(np.mat('0.4;-0.8; 0')),
                               np.array(np.mat('-1.2;0.4; 0'))],
        'goal_points': np.array(np.mat('+1.3; -0.3; 0')),
        'obs_points': np.array(np.mat('0.5 -0.8 -0.6 0.7 0.8; -0.5 0.4 -0.8 0.8 -0.8; 0 0 0 0 0'))
    },
    'robot-on-walls': {
        'initial_conditions': [
            np.array(np.mat('1.47; 0.2; 0')),
            np.array(np.mat('1.47; 0.2; 1.57')),
            np.array(np.mat('1.47; 0.2; 3.14')),
            np.array(np.mat('1.47; 0.2; -1.57'))],
        'goal_points': np.array(np.mat('-1.3; 0; 0')),
        'obs_points': np.array(np.mat('0.0 0.0 0.0; -0.8 0.8 0; 0 0 0 '))
    },
    'three-obstacles-wall':  {
        'initial_conditions': [
            np.array(np.mat('-1.0; 0.0; 0'))
        ],
        'goal_points': np.array(np.mat('1.0; 0.0; 0')),
        'obs_points': np.array(np.mat('0.0 0.0 0.0; 0.0 -0.3 0.3; 0 0 0 '))
    },
    'single-obstacle-wall':  {
        'initial_conditions': [
            np.array(np.mat('-1.0; 0.0; 0'))
        ],
        'goal_points': np.array(np.mat('1.0; 0.0; 0')),
        'obs_points': np.array(np.mat('0.0; 0.0; 0.0'))
    },
    'five-obstacles-wall':  {
        'initial_conditions': [
            np.array(np.mat('-1.0; 0.0; 0'))
        ],
        'goal_points': np.array(np.mat('1.0; 0.0; 0')),
        'obs_points': np.array(np.mat('0.0 0.0 0.0 0.0 0.0;0.6 0.3 0.0 -0.3 -0.6; 0 0 0 0 0'))
    },
    'obastacles_issue': {
        'initial_conditions': [np.array(np.mat('-1.4;0.9; 0')),
                               np.array(np.mat('-1.4;0; 0')),
                               np.array(np.mat('0;0.9; 0')),
                               np.array(np.mat('1.4;0.9; 0')),
                               np.array(np.mat('1.4;0; 0'))
                               ],
        'goal_points': np.array(np.mat('0; 0; 0')),
        'obs_points': np.array(np.mat('-0.3 -0.3 -0.3 0 0.3 0.3 0.3; -0.3 0 0.3 0.3 0.3 0 -0.3; 0 0 0 0 0 0 0'))
    },
    'goal_up_left': {
        'initial_conditions': INITIAL_CONDITIONS_BASELINE,
        'goal_points': np.array(np.mat('-1.4; 0.8; 0')),
        'obs_points': OBS_POINTS_BASELINE
    },
    'for_ioc':{
        'initial_conditions': [
            np.array(np.mat('-0.3;-0.65; 0')),
            np.array(np.mat('-0.3;0.9; 1.57')),
            np.array(np.mat('-1.40;0.625; 3.14')),
            np.array(np.mat('1.4;0.9; 0')),
            np.array(np.mat('0.2;0.9; 0')),
            np.array(np.mat('1.2;-0.8; 0')),
            np.array(np.mat('-1;0.9; 0')),
            np.array(np.mat('-1;-0.75; 0')),
            np.array(np.mat('-1.4;-0.15; 0')),
            np.array(np.mat('-1.4;-0.45; 0')),
            np.array(np.mat('-1.4;-0.85; 0')),
            np.array(np.mat('-1.4; 0.90; 0')),
            np.array(np.mat('-1.4;-0.90; 0')),
            np.array(np.mat('0.4;-0.8; 0')), #non arriva
            np.array(np.mat('-1.47; 0.0; 0')),
            np.array(np.mat('-1.47; 0.0; 3.14')),
            np.array(np.mat('-1.47;-0.25; 3.14')),
            np.array(np.mat('-1.47; 0.25; 3.14'))],
        'goal_points': np.array(np.mat('+1.3; -0.3; 0')),
        'obs_points': np.array(np.mat('0.5 -0.8 -0.5 0.7 0.8; -0.5 0.4 -0.8 0.8 -0.8; 0 0 0 0 0'))
    }
    
}

selected_scenario = 'for_ioc'
selected_cost_function = state_cost

In [ ]:
# Define goal and obstacle points
goal_points = SCENARIOS[selected_scenario]['goal_points']
obs_points = SCENARIOS[selected_scenario]['obs_points'] # (3 x n_obstacles) matrix. Each column represents an obstacle. Rows contain the (x,y,theta) of the obstacle.

# Dopo vedremo che questi 'ostacoli' virtuali si mappano a due ostacoli fisici, uno centrato in (0, -0.8) e un altro più lungo centrato in (0, 0.5).

# Define the initial states of the robots
initial_conditions = SCENARIOS[selected_scenario]['initial_conditions']

# Define the cost function
state_cost = selected_cost_function

# Define whether to show the robotarium simulations
SHOW_SIMULATIONS = True

In [ ]:
# Define the grid boundaries
X_LIMITS = [-1.5, 1.5]
Y_LIMITS = [-1, 1]
EPSILON_LIMITS = 0 # Used to plot a little bit outside the robotarium boundaries
X_SIZE = X_LIMITS[1]-X_LIMITS[0]+2*EPSILON_LIMITS
Y_SIZE = Y_LIMITS[1]-Y_LIMITS[0]+2*EPSILON_LIMITS

# Define the grid of points where to evaluate the cost function
X_axis = np.linspace(X_LIMITS[0]-EPSILON_LIMITS, X_LIMITS[1]+EPSILON_LIMITS, 300)
Y_axis = np.linspace(Y_LIMITS[0]-EPSILON_LIMITS, Y_LIMITS[1]+EPSILON_LIMITS, 200)

X_grid, Y_grid = np.meshgrid(X_axis, Y_axis)

# Evaluate the cost function on the grid
costs = np.array([[state_cost((x,y),goal_points,obs_points) for x in X_axis] for y in Y_axis] ) #TODO: speed up this line by vectorizing it
costs = costs.squeeze()

In [ ]:
%matplotlib tk
from matplotlib import cm
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 12})

# Plot the cost function
fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(projection='3d')
ax.set_xlabel('X [m]', labelpad=10)
ax.set_ylabel('Y [m]', labelpad=10)
ax.set_title('Cost function', fontdict={'fontsize': 20})

surf = ax.plot_surface(X_grid, Y_grid, costs, cmap=cm.coolwarm, linewidth=0, antialiased=False, ccount=100, rcount=100) #rcount and ccount are used to make the surface smoother
fig.colorbar(surf, shrink=0.5)
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle

fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot()
ax.set_xlabel('X [m]', labelpad=20)
ax.set_ylabel('Y [m]', labelpad=20)
ax.set_title('Cost function', fontdict={'fontsize': 20})

# Plot the heatmap of the cost function
pcolormesh = ax.pcolormesh(X_grid, Y_grid, costs, cmap='coolwarm', shading='auto', zorder=1)

# Plot the level curves of the cost function
contour = ax.contour(X_grid, Y_grid, costs, cmap='coolwarm', levels=10, linewidths=2, linestyles='dashed', zorder=2, alpha=1, vmin=0, vmax=600) #vmin and vmax are used to make the contour lines more visible

# Scatter the obstacle points
ax.scatter(obs_points[0,:], obs_points[1,:], marker='o', s=50, color='red', zorder=3)

# Add a colorbar
fig.colorbar(pcolormesh, ax=ax) 

# Create a rectangle
rectangle = Rectangle((X_LIMITS[0], Y_LIMITS[0]), X_LIMITS[1]-X_LIMITS[0], Y_LIMITS[1]-Y_LIMITS[0], fill=False, color="black", linewidth=1, zorder=3)
# Add the rectangle to the plot
ax.add_patch(rectangle)

In [ ]:
%matplotlib inline
# Calculate the gradient of the cost function
X_grad, Y_grad = np.gradient(costs.T, X_axis, Y_axis) #We need to transpose the costs matrix because we are passing the x axis as the first argument
# Calculate the module of the gradient
Z_grad = np.sqrt(X_grad**2 + Y_grad**2).T #We need to transpose the result because we need to plot it

fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot()
ax.set_xlabel('X [m]', labelpad=20)
ax.set_ylabel('Y [m]', labelpad=20)
ax.set_title('Cost function gradient module', fontdict={'fontsize': 20})

# Plot the heatmap of the module of the gradient of the cost function
pcolormesh = ax.pcolormesh(X_grid, Y_grid, Z_grad, zorder=1, cmap='RdGy', vmin=0, vmax=200)

# Add a colorbar
fig.colorbar(pcolormesh, ax=ax) 

In [ ]:
%matplotlib inline
initial_conditions = np.array(initial_conditions).squeeze()
if initial_conditions.ndim == 1:
    initial_conditions = initial_conditions.reshape((1,3))
fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot()
ax.scatter(initial_conditions[:,0],initial_conditions[:,1])
ax.scatter(goal_points[0],goal_points[1], color='green')
ax.scatter(obs_points[0,:], obs_points[1,:], marker='o', s=50, color='red', zorder=3)
ax.set_xlim(X_LIMITS)
ax.set_ylim(Y_LIMITS)
l = 0.1
for state_index, state in enumerate(initial_conditions):
    ax.arrow(state[0],state[1],l*np.cos(state[2]),l*np.sin(state[2]))
    ax.annotate(str(state_index),(state[0],state[1]))

# Restore the initial_conditions variable to the original value
initial_conditions = SCENARIOS[selected_scenario]['initial_conditions']

In [ ]:
initial_conditions = SCENARIOS[selected_scenario]['initial_conditions']
N_experiment = len(initial_conditions)
X_Si, D_Xi = robotarium_simulation(initial_conditions=initial_conditions, max_iteration=2500, show_simulation=True, cost_function=state_cost)

In [ ]:
XX = X_Si #Insieme degli stati nel sistema single-integrator (y_1, y_2) per ogni simulazione
UU = D_Xi #Insieme delle azioni di controllo per ogni simulazione

In [ ]:
#Prepare data for plotting
X = []
X_plot = []
U = []
U_plot = []

for i in range(len(XX)):
    X.append(np.array(XX[i])) #X=XX ma è un array di array numpy
    X_plot.append(np.array(XX[i])) #X_plot = XX ma è un array numpy
X = np.concatenate(X, axis=0) #X adesso è un array numpy. La prima dimensione contiene le traiettorie di tutti gli esperimenti concatenate e la secoda indicizza lo stato
print(X.shape)
X = np.reshape(X, (-1, 2)) # Rimuove l'ultima dimensione inutile
U = []
for i in range(len(UU)):
    U.append(np.array(UU[i])) #U=UU ma è un array di array numpy
    U_plot.append(np.array(UU[i])) #U_plot=UU ma è un array di array numpy

U = np.concatenate(U, axis=0) #Stessa cosa di prima per le traiettorie di ingresso
U = np.reshape(U, (-1, 2))

simulation_from_FOC=X

In [ ]:
%matplotlib inline
#Task: plot trajectories with different colors
plt.figure(figsize=(15,10))

for I in range(N_experiment):
    trajectory = np.array(X_Si[I])
    plt.plot(trajectory[:,0],trajectory[:,1])

#Add a star to the starting point of each trajectory
for I in range(N_experiment):
    trajectory = np.array(X_Si[I])
    plt.scatter(trajectory[0,0], trajectory[0,1], marker='*', s=200, color='black', zorder=3)

#Add a legend to the plot
legend = []
for i in range(N_experiment):
    legend = legend + ['Experiment {}'.format(i+1)]
    plt.legend(legend)

#Draw obstacles
for i in range(len(obs_points[0])):
    for j in range(len(obs_points[1])):
        if (i==j):
            square=plt.Rectangle((obs_points[0][i]-0.15,obs_points[1][j]-0.15), 0.3, 0.3, fc='red',ec="black")
            plt.gca().add_patch(square)

#Draw goal point
square = plt.Rectangle((float(goal_points[0][0])-0.15,float(goal_points[1][0])-0.15), 0.3, 0.3, fc='green',ec="black") # Obiettivo da raggiungere
plt.gca().add_patch(square)

plt.ylim(-1,1)
plt.xlim(-1.5,1.5)
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.savefig('Training_Trajectories.jpg',dpi=1000,bbox_inches ='tight')

## Alternative feature

In [ ]:
def base_feature(next_state,goal_points,obs_points,N_feature):
    v = np.array([0.025, 0.025], dtype=np.float32) #vettore delle varianze
    covar = np.diag(v) #matrice di varianze e covarianze
    features = np.zeros(len(obs_points[0]) + 1)
    for i in range(np.size(obs_points,axis=1)):
        features[i+1] = my_logpdf(next_state[:2],obs_points[:2,i],covar) #Modelliamo gli ostacoli come multivariate gaussiane indipendenti (cov=0) centrate nell'ostacolo ipotizzato

    features[0] = (((next_state[0]-goal_points[0])**2 + (next_state[1]-goal_points[1])**2)) #La prima feature è relativa al raggiungimento del goal point

    return features

def augmented_feature_walls(next_state,goal_points,obs_points,N_feature):
    v = np.array([0.025, 0.025], dtype=np.float32) #vettore delle varianze
    covar = np.diag(v) #matrice di varianze e covarianze
    features = np.zeros(len(obs_points[0]) + 1)
    for i in range(np.size(obs_points,axis=1)):
        features[i+1] = my_logpdf(next_state[:2],obs_points[:2,i],covar) #Modelliamo gli ostacoli come multivariate gaussiane indipendenti (cov=0) centrate nell'ostacolo ipotizzato
    features[0] = (((next_state[0]-goal_points[0])**2 + (next_state[1]-goal_points[1])**2)) #La prima feature è relativa al raggiungimento del goal point
    
    x_walls = np.linspace(X_LIMITS[0], X_LIMITS[1], 5)
    y_walls = np.linspace(Y_LIMITS[0], Y_LIMITS[1], 5)
    
    x_wall = lambda x: np.exp(-0.5*((next_state[0]-x)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) 
    y_wall = lambda y: np.exp(-0.5*((next_state[1]-y)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) 
    
    for x in x_walls:
        features = np.append(features,x_wall(x))
        
    for y in y_walls:
        features = np.append(features,y_wall(y))
    return features




FEATURES= {
    'baseline': {
        'obs_points_f': np.array(np.mat('0 0 0 0 0 0.8 0.8 0.8 0.8 0.8 -0.8 -0.8 -0.8 -0.8 -0.8;-0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8;0 0 0 0 0 0 0 0 0 0 0 0 0 0 0')),
        'N_feature': 16,
        'func': base_feature
    },
    'augmented_feature_walls': {
        'obs_points_f': np.array(np.mat('0 0 0 0 0 0.8 0.8 0.8 0.8 0.8 -0.8 -0.8 -0.8 -0.8 -0.8;-0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8;0 0 0 0 0 0 0 0 0 0 0 0 0 0 0')),
        'N_feature': 26,
        'func': augmented_feature_walls
    },
    'augmented_feature_point': {
        'obs_points_f': np.array(np.mat('0 0 0 0 0 0.8 0.8 0.8 0.8 0.8 -0.8 -0.8 -0.8 -0.8 -0.8 1.2 1.2 1.2 1.2 1.2 0.4 0.4 0.4 0.4 0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -1.2 -1.2 -1.2 -1.2 -1.2; -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ')),
        'N_feature': 36,
        'func': base_feature
    },
    'augmented_feature_point_walls': {
        'obs_points_f': np.array(np.mat('0 0 0 0 0 0.8 0.8 0.8 0.8 0.8 -0.8 -0.8 -0.8 -0.8 -0.8 1.2 1.2 1.2 1.2 1.2 0.4 0.4 0.4 0.4 0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -1.2 -1.2 -1.2 -1.2 -1.2; -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8 -0.8 -0.4 0 0.4 0.8; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ')),
        'N_feature': 46,
        'func': augmented_feature_walls
    }

}


INITIAL_CONDITIONS_BASELINE_IOC = [np.array(np.mat('-1.4;0.9; 0')),np.array(np.mat('1;0.9; 0')),np.array(np.mat('1;-0.25; 0'))]
GOAL_POINTS_BASELINE_IOC = np.array(np.mat('-1.4; -0.8; 0'))
OBS_POINTS_BASELINE_IOC = np.array(np.mat('0 0 0 0 0;0.2 0.4 0.6 0.8 -0.8;0 0 0 0 0'))

SCENARIOS_IOC = {
    'baseline': {
        'initial_conditions': INITIAL_CONDITIONS_BASELINE_IOC,
        'goal_points': GOAL_POINTS_BASELINE_IOC,
        'obs_points': OBS_POINTS_BASELINE_IOC
    },
    'more-point': {
        'initial_conditions': [np.array(np.mat('-0.6;-0.75; 0')),np.array(np.mat('-0.1;-0.7; -1.57')), np.array(np.mat('-1.2; 0.4; 0'))],
        'goal_points': np.array(np.mat('+1.3; -0.3; 0')),
        'obs_points': np.array(np.mat('0.5 -0.8 -0.5 0.7 0.8; -0.5 0.4 -0.8 0.8 -0.8; 0 0 0 0 0'))
    }
}

In [ ]:
%%capture
#REDEFINING THE OLD FUNCTION:

import cvxpy as cp
import numpy as np
import time
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib.colors import CenteredNorm
from scipy import interpolate
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np

def plt_featurt_point(obs_points_f):
    %matplotlib inline
    fig = plt.figure(figsize=(15,10))
    ax = fig.add_subplot()
    ax.scatter(obs_points_f[0,:],obs_points_f[1,:], color='red')
    ax.scatter(goal_points[0,:],goal_points[1,:], color='green')
    ax.set_ylim(-1,1)
    ax.set_xlim(-1.5,1.5)
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_title('Feature points', fontdict={'fontsize': 20})

def plt_feature_function_part(goal_points,obs_points_f,N_feature,feature):
    %matplotlib inline

    # Define the grid of points where to evaluate the feature function
    X_axis = np.linspace(X_LIMITS[0]-EPSILON_LIMITS, X_LIMITS[1]+EPSILON_LIMITS, 300)
    Y_axis = np.linspace(Y_LIMITS[0]-EPSILON_LIMITS, Y_LIMITS[1]+EPSILON_LIMITS, 200)
    X_grid, Y_grid = np.meshgrid(X_axis, Y_axis)

    # Calculate the feature function on the grid
    features_array = np.array([[feature((x,y),goal_points,obs_points_f,N_feature) for x in X_axis] for y in Y_axis] ) #TODO: speed up this line by vectorizing it


    # Plot the feature functions for the obstacles
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(1,2,1)
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_title('Feature functions (obstacles)', fontdict={'fontsize': 20})
    ax.set_xlim(X_LIMITS[0],X_LIMITS[1])
    ax.set_ylim(Y_LIMITS[0],Y_LIMITS[1])

    ax.scatter(obs_points_f[0,:],obs_points_f[1,:], color='red', zorder=3)


    Z = np.sum(features_array[:,:,1:], axis=-1)
    colormesh = ax.pcolormesh(X_grid, Y_grid, Z, shading='auto', zorder=1)
    ax.contour(X_grid, Y_grid, Z, cmap='coolwarm', levels=5, linewidths=2, linestyles='dashed', alpha=1, zorder=2) #vmin and vmax are used to make the contour lines more visible

    # Plot feature function for the goal
    ax = fig.add_subplot(1,2,2)
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_title('Feature functions (goal)', fontdict={'fontsize': 20})
    ax.set_xlim(X_LIMITS[0],X_LIMITS[1])
    ax.set_ylim(Y_LIMITS[0],Y_LIMITS[1])

    ax.scatter(goal_points[0,:],goal_points[1,:], color='green', zorder=3)


    Z = features_array[:,:,0]
    colormesh = ax.pcolormesh(X_grid, Y_grid, Z, shading='auto', zorder=1)
    ax.contour(X_grid, Y_grid, Z, cmap='coolwarm', levels=5, linewidths=2, linestyles='dashed', alpha=1, zorder=2) #vmin and vmax are used to make the contour lines more visible

def solve_inverse_optimal_control(old_simulation,N_feature,obs_points_f,goal_points,U_space_1,U_space_2,control_space_size,feature):
    X=old_simulation
    M = np.size(X,axis=0) - 1 #size è la lunghezza di tutte le simulazioni concatenate. Concatenare le simulazioni genera 4 termini spuri.
    w = cp.Variable((1,N_feature)) #Variabili decisionali. Sono tante quanto il numero di features.
    constraints = [w >= 0] #Per hp del problema.
    R = np.zeros((99,1)) #??? Mai usato
    L = [] #Termini della funzione target da minimizzare (la funzione da minimizzare sum(L))

    f_expect = np.zeros((2,20)) #???
    feature_sampled = np.zeros((N_feature,M)) #Questo è il valore atteso a sinistra in blu (tranne la moltiplicazione per w che si può portare fuori)
    PF = np.zeros((control_space_size,control_space_size,M)) #Questa è la funzione q soprasegnato indicizzata in (u_k, k) dove k è l'indice temporale

    for i in range(M): #i è l'i-esimo termine della funzione target da ottimizzare (somma da 1 a M), sto fissando x hat 

        #############################################################################################################################
        features = np.zeros((N_feature,control_space_size,control_space_size)) #Questo è il valore atteso a destra in verde (tranne la moltiplicazione per w che si può portare fuori)
        state = np.array(X[i,:]) #Get the state \hat x_k-1

        x0 = state.reshape(-1,1) #è equivalente alla flatten(), x0 è lo stato iniziale (x0[0],x0[1])
        time_step = 0.033


        pf = np.zeros((control_space_size,control_space_size)) #Initialize pf \overline q(u)

        for j in range(control_space_size):
            for k in range(control_space_size):
                next_state = model_step(state,[U_space_1[j],U_space_2[k]],time_step)
                cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
                f = st.multivariate_normal(next_state.reshape((2,)),cov) #p(x_k|\hat x_k-1, u_k) valutata per \hat x_k-1 e u_k fissati
                #next_sample = f.mean #next_sample = next_state, inutile

                N_samples = 5
                next_samples = f.rvs(N_samples) #5 campioni relativi al prossimo stato. Dimensione 5 campioni x 2 variabili di stato
                feature_sample = np.zeros((N_feature,N_samples)) # 16 feature x 5 campioni

                for m in range(N_samples):
                    feature_sample[:,m] = feature(next_samples[m,:],goal_points,obs_points_f,N_feature) #feature_sample è una matrice che tiene sulle colonne le valutazioni delle features sui campioni. Le colonne rorrispondono ai campioni.

                features[:,j,k] = np.mean(feature_sample,axis=1) #la media è fatta su tutti gli elementi della riga [i,;], ovvero per ogni feature i-esima faccio la media su tutti i campioni
                #Alla fine del ciclo features contiene la stima del valore atteso delle features per ogni possibile ingresso (j,k) con \hat x_k-1 fissato
                #quindi ho tutte le stime dei valori attesi a destra in verde nell'equazione. Fatta con la legge dei grandi numeri.
                #Calculate the DKL for each possible input, get corresponding probability
                log_DKL = np.exp(-(-f.entropy())) #Prima parte del valore atteso a destra
                pf[j,k] = log_DKL
        PF[:,:,i] = pf #Prima parte del valore atteso a destra per ogni valore di ingresso e u_k e stato x_k-1

        features = np.reshape(features,(N_feature,control_space_size**2)) # Adesso features ha come primo indice la feature e il secondo va da 0 a 8 e indica l'ingresso di controllo

        f_sampled = model_step(state,U[i+1,:],time_step) # Costuiramo la pdf p(x_k|\hat x_k-1,\hat u_k) che serve a calcolare il valore atteso a sinistra in blu
        cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
        f1 = st.multivariate_normal(f_sampled.reshape((2,)),cov) #p(x_k|\hat x_k-1,\hat u_k)
        next_samples_f1 = f1.rvs(N_samples) #stessa cosa di prima, stima con legge dei grandi numeri del expectation blu a sinistra
        feature_sample_f1 = np.zeros((N_feature,N_samples))
        for n in range(N_samples):
            feature_sample_f1[:,n] = feature(next_samples_f1[n,:],goal_points,obs_points_f,N_feature)

        feature_sampled[:,i] = np.mean(feature_sample_f1,axis=1)  # features_sampled è la stessa cosa di features fatta sul valore atteso blu. nota che u è fissata qunidi non ho come indici j e k. Ho i come indice ed è inutile.

        # Task: solve, using cvx the convex optimization problem we saw in class. To do so:
        # (i) prepare each individual term of the summation, say l;
        # (ii) sum all the elements to define the cost function
        # (iii) solve the problem
        PF2 = np.reshape(PF,(-1,M))
        # log_sum_exp prende in ingresso il vettore colonna degli elmenti da sommare
        l = -(w @ feature_sampled[:,i]) + cp.log_sum_exp(cp.reshape(w@features[:,:],(control_space_size**2,)) + cp.log(PF2[:,i])) #(i)
        L.append(l) #(ii)

    objective = cp.Minimize(cp.sum(L)) #(iii)

    prob = cp.Problem(objective)

    result = prob.solve(verbose = True)

    weights = w.value
    
    return weights,result,prob

def state_cost(state,goal_points,obs_points):
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)

    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum
    return(cost)

def state_cost_estimated(state,goal_points,obs_points,weights):
    v = np.array([0.025, 0.025], dtype=np.float32)
    covar = np.diag(v)

    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += -weights[:,i+1]*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = -weights[:,0]*((((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2))) + gauss_sum
    return(cost)

def recostruct_cost_map(goal_points,obs_points,state_cost_func,args={}):
    Cost_Map = np.zeros((300,200))
    X_axis = np.linspace(-1.5,1.5,300)
    Y_axis = np.linspace(-1,1,200)

    for i in range(200):
        for j in range(300):

            state = np.array([X_axis[j],Y_axis[i]])
            Cost_Map[j,i] = state_cost_func(state,goal_points,obs_points,**args)
    return Cost_Map,X_axis,Y_axis



# def recostruct_cost_map_estimated(goal_points,obs_points,obs_points_f,weights):
#     Cost_Map = np.zeros((300,200))
#     X_axis = np.linspace(-1.5,1.5,300)
#     Y_axis = np.linspace(-1,1,200)

#     for i in range(200):
#         for j in range(300):

#             state = np.array ([X_axis[j],Y_axis[i]])
#             Cost_Map[j,i] = state_cost_estimated(state,goal_points,obs_points_f,weights)
#     return Cost_Map,X_axis,Y_axis

# def heat_map_estimated_cost(Cost_Map,X_axis,Y_axis):
#     %matplotlib inline

#     # Transpose the data array to rotate the heatmap
#     data_rotated = np.transpose(Cost_Map) 
    
#     fig = plt.figure(figsize=(13,10))
#     ax = fig.add_subplot()
#     ax.set_xlabel('X [m]', labelpad=20)
#     ax.set_ylabel('Y [m]', labelpad=20)
#     ax.set_title('Estimated cost function', fontdict={'fontsize': 20})

#     # Plot the heatmap of the cost function
#     pcolormesh = ax.pcolormesh(X_axis, Y_axis, data_rotated, cmap='coolwarm', shading='auto', zorder=1)

#     # Plot the level curves of the cost function
#     contour = ax.contour(X_axis, Y_axis, data_rotated, cmap='coolwarm', shading='linear', levels=7, linewidths=2, linestyles='dashed', zorder=2, alpha=1, vmin=40, vmax=100) #vmin and vmax are used to make the contour lines more visible

#     # Add a colorbar
#     fig.colorbar(pcolormesh, ax=ax)

#     # Create a rectangle
#     rectangle = Rectangle((X_LIMITS[0], Y_LIMITS[0]), X_LIMITS[1]-X_LIMITS[0], Y_LIMITS[1]-Y_LIMITS[0], fill=False, color="black", linewidth=1, zorder=3)
#     # Add the rectangle to the plot
#     ax.add_patch(rectangle)
    
#     plt.scatter(obs_points_f[0,:],obs_points_f[1,:], color='red', zorder=3)
#     plt.scatter(goal_points[0,:],goal_points[1,:], color='green', zorder=3)

#     plt.show()
    
def heat_map_cost(cost,X_axis,Y_axis,X_LIMITS,Y_LIMITS):
    %matplotlib inline
    
    fig = plt.figure(figsize=(13,10))
    ax = fig.add_subplot()
    ax.set_xlabel('X [m]', labelpad=20)
    ax.set_ylabel('Y [m]', labelpad=20)
    ax.set_title('Estimated cost function', fontdict={'fontsize': 20})

    # Plot the heatmap of the cost function
    pcolormesh = ax.pcolormesh(X_axis, Y_axis, cost, cmap='coolwarm', shading='auto', zorder=1)

    # Plot the level curves of the cost function
    contour = ax.contour(X_axis, Y_axis, cost, cmap='coolwarm', shading='linear', levels=7, linewidths=2, linestyles='dashed', zorder=2, alpha=1, vmin=40, vmax=100) #vmin and vmax are used to make the contour lines more visible

    # Add a colorbar
    fig.colorbar(pcolormesh, ax=ax)

    # Create a rectangle
    rectangle = Rectangle((X_LIMITS[0], Y_LIMITS[0]), X_LIMITS[1]-X_LIMITS[0], Y_LIMITS[1]-Y_LIMITS[0], fill=False, color="black", linewidth=1, zorder=3)
    # Add the rectangle to the plot
    ax.add_patch(rectangle)
    return plt

def add_goal_obs_heatmpap_cost(plt, obs_points, goal_points):
    plt.scatter(obs_points[0,:],obs_points[1,:], color='red', zorder=3)
    plt.scatter(goal_points[0,:],goal_points[1,:], color='green', zorder=3)
    return plt

def plot_3d_cost(cost,X_axis,Y_axis):

    fig = plt.figure(figsize=(13,13))
    ax = fig.add_subplot(projection='3d')
    ax.set_xlabel('X [m]', labelpad=10)
    ax.set_ylabel('Y [m]', labelpad=10)
    ax.set_zlabel('Cost')
    ax.set_title('Estimated Cost Function', fontdict={'fontsize': 20})

    surf = ax.plot_surface(X_grid, Y_grid, cost, cmap=cm.coolwarm, linewidth=0, antialiased=False, ccount=100, rcount=100) #rcount and ccount are used to make the surface smoother
    fig.colorbar(surf, shrink=0.5)
    plt.show()

def calculate_interpolated_cost(X_LIMITS,Y_LIMITS,goal_points,obs_points_f,weights):
    X_axis_interpolation = np.linspace(X_LIMITS[0]-0.5, X_LIMITS[1]+0.5, 600)
    Y_axis_interpolation = np.linspace(Y_LIMITS[0]-0.5, Y_LIMITS[1]+0.5, 400)
    X_grid_interpolation, Y_grid_interpolation = np.meshgrid(X_axis_interpolation, Y_axis_interpolation)
    Z = np.zeros((600,400))
    for i in range(600):
        for j in range(400):
            Z[i,j] = state_cost_estimated(np.array([X_axis_interpolation[i],Y_axis_interpolation[j]]),goal_points,obs_points_f,weights)


    interpolated_cost = interpolate.LinearNDInterpolator(np.stack((X_grid_interpolation.flatten(),Y_grid_interpolation.flatten()),axis=1),Z.T.flatten())
    return interpolated_cost ,X_axis_interpolation, Y_axis_interpolation,X_grid_interpolation, Y_grid_interpolation

def plt_interpolated_cost(interpolated_cost ,X_axis_interpolation, Y_axis_interpolation,X_grid_interpolation, Y_grid_interpolation):
    fig = plt.figure(figsize=(13,10))
    ax = fig.add_subplot()
    ax.set_xlabel('X [m]', labelpad=20)
    ax.set_ylabel('Y [m]', labelpad=20)
    ax.set_title('Interpolated cost function', fontdict={'fontsize': 20})
    TEST = np.zeros((600,400))

    for i in range(600):
        for j in range(400):
            TEST[i,j] = interpolated_cost(X_axis_interpolation[i], Y_axis_interpolation[j])
    pcolormesh = ax.pcolormesh(X_grid_interpolation, Y_grid_interpolation, TEST.T, cmap='coolwarm',  zorder=1)

#redifne control state for ioc with interpolated cost
def Control_step(state,U_space_1,U_space_2,goal_points,obs_points,cost_func):
    ###
    # Perform a control step given the fact that the target pf is uniform.
    # The function first gets the target pf (uniform) and then applies the control solution we saw in class
    
    target_pf = 1/control_space_size**2 # Uniform pf q(u_k|x_k-1)
    time_step = 0.033 # The Robotarium time-step

    pf = np.zeros((control_space_size,control_space_size))
    for i in range(control_space_size):
        for j in range(control_space_size):
            next_state = model_step(state,[U_space_1[i],U_space_2[j]],time_step)
            cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
            f = st.multivariate_normal(next_state.reshape((2,)),cov)
            N_samples = 20
            next_sample = f.rvs(N_samples) 

            # Task: what do the next three lines do?
            # Calcola una stima del valore atteso del costo nello stato successivo rispetto alla funzione f(x_k|x_k-1,u_k) usando 20 campioni
            cost=0
            for k in range(N_samples):
                #cost += state_cost_estimated(next_sample[k,:],goal_points,obs_points_f,weights)/N_samples
                cost += cost_func(next_sample[k,:])/N_samples
            log_DKL = np.exp(f.entropy()-cost).item() #item is used to convert the result to a scalar
            pf[i,j] = log_DKL 
    # Task: obtain the normalizer for the policy, call it S2
    S2 = np.sum(pf)
    # Task: obtain the normalized pf (call the variable pf)
    pf = np.divide(pf,S2)

    # Adesso abbiamo p(u_k|x_{k-1}) e facciamo il sampling della prossima azione di controllo
    # This is a trick to properly sample from the multi-dimensional pf
    flat = pf.flatten()
    sample_index = np.random.choice(a=flat.size, p=flat)
    # Take this index and adjust it so it matches the original array
    adjusted_index = np.unravel_index(sample_index, pf.shape) #Indice dell'azione

    #Formatta l'azione come vettore colonna
    action = np.reshape(np.array([U_space_1[adjusted_index[0]],U_space_2[adjusted_index[1]]]),(2,1))
    return(action)

def manipulation_simulation_data(X_Si, D_Xi):
    XX = X_Si
    UU = D_Xi
    X = []
    X_plot = []
    U = []
    U_plot = []

    for i in range(len(XX)):
        X.append(np.array(XX[i]))
        X_plot.append(np.array(XX[i]))

    X = np.concatenate(X, axis=0)
    X = np.reshape(X, (-1, 2))

    U = []
    for i in range(len(UU)):
        U.append(np.array(UU[i]))
        U_plot.append(np.array(UU[i]))

    U = np.concatenate(U, axis=0)
    U = np.reshape(U, (-1, 2))
    return X,U

def plot_trajectory(X_Si, N_experiment, obs_points, goal_points):
    %matplotlib inline
    #Task: plot trajectories with different colors
    plt.figure(figsize=(15,10))

    for I in range(N_experiment):
        trajectory = np.array(X_Si[I])
        plt.plot(trajectory[:,0],trajectory[:,1])

    # Add a star to the starting point of each trajectory
    for I in range(N_experiment):
        trajectory = np.array(X_Si[I])
        plt.scatter(trajectory[0,0], trajectory[0,1], marker='*', s=200, color='black')

    # Add a legend to the plot
    legend = []
    for i in range(N_experiment):
        legend = legend + ['Experiment {}'.format(i+1)]
        plt.legend(legend)


    #Draw obstacles
    for i in range(len(obs_points[0])):
        for j in range(len(obs_points[1])):
            if (i==j):
                square=plt.Rectangle((obs_points[0][i]-0.15,obs_points[1][j]-0.15), 0.3, 0.3, fc='red',ec="black")
                plt.gca().add_patch(square)

    #Draw goal point
    square = plt.Rectangle((float(goal_points[0][0])-0.15,float(goal_points[1][0])-0.15), 0.3, 0.3, fc='green',ec="black") # Obiettivo da raggiungere
    plt.gca().add_patch(square)

    #plt.scatter(obs_points_f[0,:],obs_points_f[1,:])

    plt.ylim(-1,1)
    plt.xlim(-1.5,1.5)
    plt.xlabel('X [m]')
    plt.ylabel('Y [m]')
    #plt.show()
    return plt

def add_feature_plot_trajectory(plt, obs_points_f):
    plt.scatter(obs_points_f[0,:],obs_points_f[1,:])
    return plt

def plt_feature_point_weight(weights, obs_points_f):
    %matplotlib inline

    print(weights)
    fig, ax = plt.subplots()
    ax.scatter(obs_points_f[0,:],obs_points_f[1,:])
    for i, w in enumerate(weights[0,1:]):
        ax.annotate(round(w,2), (obs_points_f[0,i] + 0.02, obs_points_f[1,i] + 0.02))

    # Set x-axis limits
    ax.set_xlim([-1.5, 1.5])

    # Set y-axis limits
    ax.set_ylim([-1, 1])

In [ ]:
def solve_ioc_problem(selected_feature,selected_scenario_ioc):
    feature = FEATURES[selected_feature]['func']
    obs_points_f = FEATURES[selected_feature]['obs_points_f']
    N_feature = FEATURES[selected_feature]['N_feature'] 
    print("feature", feature)
    goal_points = SCENARIOS_IOC[selected_scenario_ioc]['goal_points']
    obs_points = SCENARIOS_IOC[selected_scenario_ioc]['obs_points'] # (3 x n_obstacles) matrix. Each column represents an obstacle. Rows contain the (x,y,theta) of the obstacle.
    initial_conditions = SCENARIOS_IOC[selected_scenario_ioc]['initial_conditions']

    plt_featurt_point(obs_points_f)
    plt_feature_function_part(goal_points,obs_points_f,N_feature,feature)
    weights,result,prob= solve_inverse_optimal_control(simulation_from_FOC,N_feature,obs_points_f,goal_points,U_space_1,U_space_2,control_space_size,feature)

    
    Cost_Map_original,X_axis_original,Y_axis_original = recostruct_cost_map(goal_points,obs_points, state_cost)
    Cost_Map,X_axis,Y_axis=recostruct_cost_map(goal_points,obs_points_f, state_cost_estimated, {"weights": weights})
    plt=heat_map_cost(np.transpose(Cost_Map),X_axis,Y_axis,X_LIMITS,Y_LIMITS)
    add_goal_obs_heatmpap_cost(plt, obs_points_f, goal_points)
    #plt_3d_estimated_cost(Cost_Map,X_axis,Y_axis)
    plot_3d_cost(np.transpose(Cost_Map) ,X_axis,Y_axis)
    interpolated_cost ,X_axis_interpolation, Y_axis_interpolation,X_grid_interpolation, Y_grid_interpolation = calculate_interpolated_cost(X_LIMITS,Y_LIMITS,goal_points,obs_points_f,weights)
    plt_interpolated_cost(interpolated_cost ,X_axis_interpolation, Y_axis_interpolation,X_grid_interpolation, Y_grid_interpolation)
    N_experiment = len(initial_conditions)
    X_Si, D_Xi= robotarium_simulation(initial_conditions=initial_conditions, max_iteration=25000, show_simulation=True, cost_function=interpolated_cost)
    X,U= manipulation_simulation_data(X_Si, D_Xi)
    plt=plot_trajectory(X_Si, N_experiment, obs_points, goal_points)
    add_feature_plot_trajectory(plt, obs_points_f)
    plt_feature_point_weight(weights, obs_points_f)


In [ ]:
selected_feature = 'baseline'
selected_scenario_ioc = 'more-point'
solve_ioc_problem(selected_feature,selected_scenario_ioc)

### Adding feature point

In [ ]:
selected_feature = 'augmented_feature_point'
selected_scenario_ioc = 'more-point'
solve_ioc_problem(selected_feature,selected_scenario_ioc)

The thickening of the featur point grid, allows us to more accurately reconstruct the cost associated with obstacles that are not in exact correspondence with a featur point.
This improvement cannot be taken to the extreme, as the optimization problem would grow out of control.

### Adding boundiers term

In [ ]:
selected_feature = 'augmented_feature_walls'
selected_scenario_ioc = 'more-point'
solve_ioc_problem(selected_feature,selected_scenario_ioc)

As noted by the previous case, the thickening of feature points allows for more accurate construction of obstacles in the robotarium, but not for reconstructing walls.
Therefore, additional features are exhibited in which using univariate Gaussians arranged uniformly in the robotarium, we try to reconstruct just the walls.

### Adding feature point and boundiers term

In [ ]:
selected_feature = 'augmented_feature_point_walls'
selected_scenario_ioc = 'more-point'
solve_ioc_problem(selected_feature,selected_scenario_ioc)

Ultimately, the two previous improvements were combined so that both problems that emerged previously were solved